In [4]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# read_tables

In [161]:
df = pd.read_csv('../protein_positions.tsv',sep='\t')
df.head()

,Uniprot_AC,Pos_A,Pos_C
0,P63279,106,74
1,P63279,106,67
2,P63279,106,78
3,P63279,106,40
4,P63279,26,106


In [162]:
df.loc[df['Uniprot_AC']=='BLAT_ECOLX', 'Uniprot_AC'] = 'P62593'
df.loc[df['Uniprot_AC']=='MCP2_ECOLI', 'Uniprot_AC'] = 'P07017'
df.loc[df['Uniprot_AC']=='P53_HUMAN', 'Uniprot_AC'] = 'P04637'
df.loc[df['Uniprot_AC']=='PMY13_CAEEL', 'Uniprot_AC'] = 'O18209'


In [385]:
struct = pd.read_csv('paired_PMD_full_annotation_table.tsv',sep='\t')
struct['Input-ID'] = struct['Protein-ID']
struct1 = pd.read_csv('maveDB_full_annotation_table.tsv',sep='\t')
struct = struct.append(struct1)
struct['Input-ID'] = struct['Input-ID'].str.split(':',1).str[0]
struct.head()

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
0,A992881:1:[DIRECT],365,4X0F,A,586,F,0.972010,0.837953,3.22,Protein interaction,B,A:804,A992881
1,A992881:1:[DIRECT],354,4X0F,A,575,Y,0.972010,0.837953,3.22,ligand interaction,NaN,A:804,A992881
2,A992881:1:[DIRECT],401,4X0F,A,622,I,0.972010,0.837953,3.22,Protein interaction,B,NaN,A992881
3,A992881:1:[DIRECT],383,4X0F,A,604,A,0.972010,0.837953,3.22,Core,NaN,NaN,A992881
4,A992881:1:[DIRECT],365,4WZI,B,586,F,0.971867,0.807851,2.58,Protein interaction,A,NaN,A992881


In [268]:
import re

In [386]:
struct.loc[struct['Input-ID']=='BLAT_ECOLX', 'Input-ID'] = 'P62593'
struct.loc[struct['Input-ID']=='MCP2_ECOLI', 'Input-ID'] = 'P07017'
struct.loc[struct['Input-ID']=='P53_HUMAN', 'Input-ID'] = 'P04637'
struct.loc[struct['Input-ID']=='PMY13_CAEEL', 'Input-ID'] = 'O18209'



In [269]:
def find_number(s):
    s = re.findall(r'\d+', s)
    if len(s)>0:
        return int(s[0])
    
    else:
        return s

In [270]:
struct['Residue-ID'] = struct['Residue-ID'].apply(find_number)

In [166]:
def find_pdb(Uniprot_AC,Pos_A, Pos_C):
    local_A = struct[(struct['Input-ID']==Uniprot_AC)&(struct[
                  'Position']==Pos_A)]
    local_C = struct[(struct['Input-ID']==Uniprot_AC)&(struct[
                  'Position']==Pos_C)]
    pdb_A = set(local_A['PDB-ID'].values.tolist())
    pdb_C = set(local_C['PDB-ID'].values.tolist())
    joined = list(pdb_A&pdb_C)
    
    if len(joined) == 0:
        return '?'
    
    elif len(joined) == 1:
        return joined[0]
    
    else:
        local_A = local_A[local_A['PDB-ID'].isin(joined)]
        local_A = local_A.sort_values(by=['Sequence identity'],ascending=False)
        local_A = local_A.reset_index(drop=True)
        return local_A.loc[0,'PDB-ID']
        
    

In [167]:
#?df.sort_values

In [168]:
df['PDB_ID'] = df.apply(
        lambda x: find_pdb(x.Uniprot_AC, x.Pos_A,x.Pos_C),axis=1)

In [169]:
df.to_csv('../protein_positions_with_max_pdb.tsv',sep='\t',index=False)

# add structures where no common PDB_ID structure

In [170]:
df = pd.read_csv('../protein_positions_with_max_pdb.tsv',sep='\t')
df.head()

,Uniprot_AC,Pos_A,Pos_C,PDB_ID
0,P63279,106,74,1A3S
1,P63279,106,67,1A3S
2,P63279,106,78,1A3S
3,P63279,106,40,1A3S
4,P63279,26,106,1A3S


In [16]:
df['Pos_A']

0        106
1        106
2        106
3        106
4         26
        ... 
22424     30
22425    159
22426     42
22427     82
22428    247
Name: Pos_A, Length: 22429, dtype: int64

In [171]:
struct = pd.read_csv('paired_PMD_full_annotation_table.tsv',sep='\t')
struct['Input-ID'] = struct['Protein-ID']
struct1 = pd.read_csv('maveDB_full_annotation_table.tsv',sep='\t')
struct = struct.append(struct1)
struct['Input-ID'] = struct['Input-ID'].str.split(':',1).str[0]
struct.head()


struct.loc[struct['Input-ID']=='BLAT_ECOLX', 'Input-ID'] = 'P62593'
struct.loc[struct['Input-ID']=='MCP2_ECOLI', 'Input-ID'] = 'P07017'
struct.loc[struct['Input-ID']=='P53_HUMAN', 'Input-ID'] = 'P04637'
struct.loc[struct['Input-ID']=='PMY13_CAEEL', 'Input-ID'] = 'O18209'




In [17]:
struct['Position']

0         365
1         354
2         401
3         383
4         365
         ... 
118711     53
118712    370
118713    367
118714    333
118715     45
Name: Position, Length: 626721, dtype: int64

In [172]:
def find_pdb_single(Uniprot_AC,Pos, PDB_ID):
    if PDB_ID != '?':
        return PDB_ID
    
    else:
        local_A = struct[(struct['Input-ID']==Uniprot_AC)&(struct[
                  'Position']==Pos)]
        if len(local_A) > 0:
            
            local_A = local_A.sort_values(by=['Sequence identity'],ascending=False)
            local_A = local_A.reset_index(drop=True)
            return local_A.loc[0,'PDB-ID']
        else:
            return PDB_ID

In [173]:
df['PDB_ID_A'] = df.apply(
        lambda x: find_pdb_single(x.Uniprot_AC, x.Pos_A,x.PDB_ID),axis=1)

In [174]:
df['PDB_ID_C'] = df.apply(
        lambda x: find_pdb_single(x.Uniprot_AC, x.Pos_C,x.PDB_ID),axis=1)

In [175]:
df.to_csv('../protein_positions_with_max_pdb.tsv',sep='\t',index=False)

In [176]:
df[df['PDB_ID_A']=='?']

,Uniprot_AC,Pos_A,Pos_C,PDB_ID,PDB_ID_A,PDB_ID_C
1797,P38398,125,134,?,?,7JZV
1803,P38398,115,134,?,?,7JZV
1813,P38398,126,194,?,?,7JZV
1833,P38398,110,224,?,?,7JZV
1857,P38398,253,246,?,?,?
...,...,...,...,...,...,...
22409,A004640,398,338,?,?,?
22410,A940619,124,121,?,?,?
22412,A890094,14,5,?,?,?
22426,A015899,42,18,?,?,?


In [177]:
len(df[df['PDB_ID_C']=='?'])

1902

In [178]:
df[df['PDB_ID_C']=='?']

,Uniprot_AC,Pos_A,Pos_C,PDB_ID,PDB_ID_A,PDB_ID_C
1820,P38398,221,175,?,7JZV,?
1845,P38398,224,164,?,7JZV,?
1857,P38398,253,246,?,?,?
1861,P38398,251,253,?,?,?
1866,P38398,139,253,?,7JZV,?
...,...,...,...,...,...,...
22409,A004640,398,338,?,?,?
22410,A940619,124,121,?,?,?
22412,A890094,14,5,?,?,?
22426,A015899,42,18,?,?,?


In [29]:
struct.head()

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
0,A992881:1:[DIRECT],365,4X0F,A,586,F,0.972010,0.837953,3.22,Protein interaction,B,A:804,A992881
1,A992881:1:[DIRECT],354,4X0F,A,575,Y,0.972010,0.837953,3.22,ligand interaction,NaN,A:804,A992881
2,A992881:1:[DIRECT],401,4X0F,A,622,I,0.972010,0.837953,3.22,Protein interaction,B,NaN,A992881
3,A992881:1:[DIRECT],383,4X0F,A,604,A,0.972010,0.837953,3.22,Core,NaN,NaN,A992881
4,A992881:1:[DIRECT],365,4WZI,B,586,F,0.971867,0.807851,2.58,Protein interaction,A,NaN,A992881


# merge structman info

In [179]:
df = pd.read_csv('../protein_positions_with_max_pdb.tsv',sep='\t')

df.head()

,Uniprot_AC,Pos_A,Pos_C,PDB_ID,PDB_ID_A,PDB_ID_C
0,P63279,106,74,1A3S,1A3S,1A3S
1,P63279,106,67,1A3S,1A3S,1A3S
2,P63279,106,78,1A3S,1A3S,1A3S
3,P63279,106,40,1A3S,1A3S,1A3S
4,P63279,26,106,1A3S,1A3S,1A3S


In [180]:
df.drop([ 'PDB_ID'], axis=1, inplace=True)

In [181]:
df = pd.merge(df, struct, left_on=['Uniprot_AC', 'Pos_A', 'PDB_ID_A'],
        right_on = ['Input-ID', 'Position','PDB-ID'], how='left')

In [182]:
df[(df['PDB_ID_A']!='?')&(df['Residue RIN simple classification'].isna())]

,Uniprot_AC,Pos_A,Pos_C,PDB_ID_A,PDB_ID_C,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID


In [183]:
df.tail()

,Uniprot_AC,Pos_A,Pos_C,PDB_ID_A,PDB_ID_C,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
22772,A941188,159,160,7JY6,7JY6,A941188:1:RECA_SHIFL,159.0,7JY6,A,158,E,1.0,0.943343,2.5,Surface,NaN,NaN,A941188
22773,A941188,159,160,7JY6,7JY6,A941188:1:RECA_SHIFL,159.0,7JY6,I,158,E,1.0,0.849858,2.5,Protein interaction,H,NaN,A941188
22774,A015899,42,18,?,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22775,A930848,82,45,2GN5,2GN5,A930848:1:VHED_BPFD,82.0,2GN5,A,82,R,1.0,1.000000,2.3,Surface,NaN,NaN,A930848
22776,A005229,247,274,?,?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
df.columns

Index(['Uniprot_AC', 'Pos_A', 'Pos_C', 'PDB_ID_A', 'PDB_ID_C', 'Protein-ID',
       'Position', 'PDB-ID', 'Chain', 'Residue-ID', 'Residue AA type',
       'Sequence identity', 'Coverage', 'Resolution',
       'Residue RIN simple classification', 'Chain type interaction partners',
       'Small molecules type interaction partners', 'Input-ID'],
      dtype='object')

In [185]:
df.drop([ 'Input-ID', 'Position', 'PDB-ID'], axis=1, inplace=True)

df = df.rename(columns={'Chain':'Chain_A', 'Residue-ID':'Residue-ID_A', 
                        'Residue AA type':'PDB_AA_A',
       'Sequence identity':'Sequence_identity_A', 
                        'Coverage':'Coverage_A', 'Resolution':'Resolution_A',
       'Residue RIN simple classification':'RIN_A', 
        'Chain type interaction partners':'Chain_partners_A',
       'Small molecules type interaction partners':
                        'Small_molecules_partners_A'})

In [186]:
df = pd.merge(df, struct, left_on=['Uniprot_AC', 'Pos_C', 'PDB_ID_C'],
        right_on = ['Input-ID', 'Position','PDB-ID'], how='left')



df.drop([ 'Input-ID', 'Position', 'PDB-ID'], axis=1, inplace=True)

df = df.rename(columns={'Chain':'Chain_C', 'Residue-ID':'Residue-ID_C', 
                        'Residue AA type':'PDB_AA_C',
       'Sequence identity':'Sequence_identity_C', 
                        'Coverage':'Coverage_C', 'Resolution':'Resolution_C',
       'Residue RIN simple classification':'RIN_C', 
        'Chain type interaction partners':'Chain_partners_C',
       'Small molecules type interaction partners':
                        'Small_molecules_partners_C'})

In [187]:
df = df.fillna('?')

In [188]:
df

,Uniprot_AC,Pos_A,Pos_C,PDB_ID_A,PDB_ID_C,Protein-ID_x,Chain_A,Residue-ID_A,PDB_AA_A,Sequence_identity_A,Coverage_A,Resolution_A,RIN_A,Chain_partners_A,Small_molecules_partners_A,Protein-ID_y,Chain_C,Residue-ID_C,PDB_AA_C,Sequence_identity_C,Coverage_C,Resolution_C,RIN_C,Chain_partners_C,Small_molecules_partners_C
0,P63279,106,74,1A3S,1A3S,P63279,A,106,A,1,1,2.8,Surface,?,?,P63279,A,74,K,1,1,2.8,Surface,?,?
1,P63279,106,67,1A3S,1A3S,P63279,A,106,A,1,1,2.8,Surface,?,?,P63279,A,67,D,1,1,2.8,Surface,?,?
2,P63279,106,78,1A3S,1A3S,P63279,A,106,A,1,1,2.8,Surface,?,?,P63279,A,78,E,1,1,2.8,Surface,?,?
3,P63279,106,40,1A3S,1A3S,P63279,A,106,A,1,1,2.8,Surface,?,?,P63279,A,40,N,1,1,2.8,Surface,?,?
4,P63279,26,106,1A3S,1A3S,P63279,A,26,A,1,1,2.8,Core,?,?,P63279,A,106,A,1,1,2.8,Surface,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23560,A941188,159,160,7JY6,7JY6,A941188:1:RECA_SHIFL,I,158,E,1,0.849858,2.5,Protein interaction,H,?,A941188:1:RECA_SHIFL,A,159,I,1,0.943343,2.5,Surface,?,?
23561,A941188,159,160,7JY6,7JY6,A941188:1:RECA_SHIFL,I,158,E,1,0.849858,2.5,Protein interaction,H,?,A941188:1:RECA_SHIFL,I,159,I,1,0.849858,2.5,Protein interaction,H,?
23562,A015899,42,18,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
23563,A930848,82,45,2GN5,2GN5,A930848:1:VHED_BPFD,A,82,R,1,1,2.3,Surface,?,?,A930848:1:VHED_BPFD,A,45,V,1,1,2.3,Surface,?,?


In [189]:
df.to_csv('../protein_positions_with_max_pdb.tsv',sep='\t',index=False)

# find Recommended structure

In [190]:
df.drop([ 'Protein-ID_y', 'Protein-ID_x'], axis=1, inplace=True)

df.columns

Index(['Uniprot_AC', 'Pos_A', 'Pos_C', 'PDB_ID_A', 'PDB_ID_C', 'Chain_A',
       'Residue-ID_A', 'PDB_AA_A', 'Sequence_identity_A', 'Coverage_A',
       'Resolution_A', 'RIN_A', 'Chain_partners_A',
       'Small_molecules_partners_A', 'Chain_C', 'Residue-ID_C', 'PDB_AA_C',
       'Sequence_identity_C', 'Coverage_C', 'Resolution_C', 'RIN_C',
       'Chain_partners_C', 'Small_molecules_partners_C'],
      dtype='object')

In [191]:
for elem in ['PDB_ID_A', 'PDB_ID_C', 'Chain_A',
       'Residue-ID_A', 'PDB_AA_A', 'Sequence_identity_A', 'Coverage_A',
       'Resolution_A', 'RIN_A', 'Chain_partners_A',
       'Small_molecules_partners_A', 'Chain_C', 'Residue-ID_C', 'PDB_AA_C',
       'Sequence_identity_C', 'Coverage_C', 'Resolution_C', 'RIN_C',
       'Chain_partners_C', 'Small_molecules_partners_C']:
    
    df = df.rename(columns={elem:'Max_'+elem})
df.head()
    

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_partners_A,Max_Small_molecules_partners_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Max_Chain_partners_C,Max_Small_molecules_partners_C
0,P63279,106,74,1A3S,1A3S,A,106,A,1,1,2.8,Surface,?,?,A,74,K,1,1,2.8,Surface,?,?
1,P63279,106,67,1A3S,1A3S,A,106,A,1,1,2.8,Surface,?,?,A,67,D,1,1,2.8,Surface,?,?
2,P63279,106,78,1A3S,1A3S,A,106,A,1,1,2.8,Surface,?,?,A,78,E,1,1,2.8,Surface,?,?
3,P63279,106,40,1A3S,1A3S,A,106,A,1,1,2.8,Surface,?,?,A,40,N,1,1,2.8,Surface,?,?
4,P63279,26,106,1A3S,1A3S,A,26,A,1,1,2.8,Core,?,?,A,106,A,1,1,2.8,Surface,?,?


In [192]:
struct['Residue RIN simple classification'] = np.where(
    struct['Residue RIN simple classification'] == 'Peptide interaction', 1,
    struct['Residue RIN simple classification'])

In [193]:
struct['Position'] = struct['Position'].astype(int)
df['Pos_A'] = df['Pos_A'].astype(int)
df['Pos_C'] = df['Pos_C'].astype(int)

In [194]:
def find_pdb(Uniprot_AC, Pos_A, Pos_C):

    local_A = struct[(struct['Input-ID'] == Uniprot_AC)
                     & (struct['Position'] == Pos_A)]
    local_C = struct[(struct['Input-ID'] == Uniprot_AC)
                     & (struct['Position'] == Pos_C)]
    pdb_A = set(local_A['PDB-ID'].values.tolist())
    pdb_C = set(local_C['PDB-ID'].values.tolist())
    joined = list(pdb_A&pdb_C)

    if len(joined) == 0:
        return '?'

    elif len(joined) == 1:
        return joined[0]

    else:
        local_A = local_A[local_A['PDB-ID'].isin(joined)]
        local_C = local_C[local_C['PDB-ID'].isin(joined)]
        local_joined = pd.merge(
            local_A,
            local_C,
            on=['PDB-ID', 'Residue RIN simple classification'],
            how='inner')
        local_joined = local_joined[(local_joined['Sequence identity_x'] > 0.7)
                                    & (local_joined['Resolution_x'] <= 2.5)]
        if len(local_joined) == 0:
            return 'bad_struct'

        local_joined = local_joined.sort_values(
            by=['Sequence identity_x', 'Coverage_x'], ascending=False)

        high_priority = [
            'Protein interaction',
            'ligand interaction',
            'Peptide interaction',
            'DNA interaction',
            'metal interaction',
            'RNA interaction',
            'ion interaction',
        ]

        local_joined1 = local_joined[local_joined[
            'Residue RIN simple classification'].isin(high_priority)]

        if len(local_joined1['Residue RIN simple classification'].unique()
               ) > 1:
            print(Uniprot_AC, Pos_A, Pos_C)
            return 'inconsistence'

        elif len(local_joined1) > 0:
            local_joined1 = local_joined1.sort_values(
                by=['Sequence identity_x', 'Coverage_x'], ascending=False)
            return local_joined1['PDB-ID'].iloc[0]

        else:
            if len(local_joined['Residue RIN simple classification'].unique()
                   ) > 1:
                print(Uniprot_AC, Pos_A, Pos_C)
                return 'inconsistence'
            else:
                return local_joined['PDB-ID'].iloc[0]

In [195]:
df['PDB-ID'] = df.apply(
        lambda x: find_pdb(x.Uniprot_AC, x.Pos_A,x.Pos_C,),axis=1)

P63279 51 61
P63279 52 117
P63279 145 136
P63279 25 42
P63279 125 28
P63279 125 52
P63279 136 5
P63279 93 136
P63279 8 136
P63279 72 96
P63279 8 97
P63279 97 37
P63279 136 155
P63279 87 123
P46937 188 190
P46937 190 188
P46937 173 202
P46937 202 174
P46937 202 177
P46937 202 173
P46937 173 174
P46937 174 173
P46937 197 188
P46937 197 190
P46937 190 197
P46937 188 197
P46937 173 177
P46937 177 174
Q9ES00 1132 1096
Q9ES00 1128 1099
Q9ES00 1128 1111
Q9ES00 1099 1128
Q9ES00 1111 1099
Q9ES00 1099 1111
Q9ES00 1151 1096
Q9ES00 1138 1157
Q9ES00 1157 1099
Q9ES00 1157 1151
Q9ES00 1154 1165
Q9ES00 1111 1165
Q9ES00 1136 1165
Q9ES00 1165 1128
Q9ES00 1128 1165
Q9ES00 1165 1154
Q9ES00 1132 1165
Q9ES00 1136 1111
Q9ES00 1136 1128
Q9ES00 1136 1099
Q9ES00 1136 1138
Q9ES00 1138 1099
Q9ES00 1138 1111
Q9ES00 1111 1138
Q9ES00 1138 1136
Q9ES00 1138 1128
Q9ES00 1136 1154
Q9ES00 1138 1154
Q9ES00 1154 1111
Q9ES00 1154 1099
Q9ES00 1111 1154
Q9ES00 1154 1128
Q9ES00 1099 1154
Q9ES00 1128 1154
P06654 254 232
P06654 

A900718 195 194
A983677 200 199
A920760 357 321
A890461 527 416
A910382 527 416
A984356 67 63
A015888 30 20
A941323 64 102
A921419 128 131
A921436 73 109
A952185 58 109
A952186 73 109
A900667 1097 1042
A900667 1097 1042
A900667 1097 1042
A900667 1097 1042
A981719 570 572
A890344 143 317
A860144 59 32
A952296 154 146
A921436 73 58
A940909 498 443


In [196]:
df.to_csv('../protein_positions_with_max__rec0_pdb.tsv',sep='\t', index=False)

In [197]:
df[df['Uniprot_AC'].isin(['O18209', 'P07017'])]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_partners_A,Max_Small_molecules_partners_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Max_Chain_partners_C,Max_Small_molecules_partners_C,PDB-ID
20465,P07017,550,398,?,1QU7,?,?,?,?,?,?,?,?,?,A,400,A,0.837004,0.410488,2.6,Core,?,?,?
20466,P07017,550,398,?,1QU7,?,?,?,?,?,?,?,?,?,B,400,A,0.832579,0.399638,2.6,Core,?,?,?
20467,P07017,550,278,?,3ZX6,?,?,?,?,?,?,?,?,?,A,67,G,0.745033,0.545126,2.65,Surface,?,?,?
20468,P07017,550,278,?,3ZX6,?,?,?,?,?,?,?,?,?,B,67,G,0.745033,0.545126,2.65,Surface,?,?,?
20469,P07017,550,334,?,1QU7,?,?,?,?,?,?,?,?,?,A,336,T,0.837004,0.410488,2.6,Core,B,?,?
20470,P07017,550,334,?,1QU7,?,?,?,?,?,?,?,?,?,B,336,T,0.832579,0.399638,2.6,Protein interaction,A,?,?
20475,O18209,560,103,?,3P1A,?,?,?,?,?,?,?,?,?,A,105,F,0.449275,0.404692,1.7,Core,?,?,?
20476,O18209,560,245,?,3P1A,?,?,?,?,?,?,?,?,?,A,250,G,0.449275,0.404692,1.7,Core,?,?,?
20477,O18209,560,163,?,3P1A,?,?,?,?,?,?,?,?,?,A,167,H,0.449275,0.404692,1.7,Surface,?,?,?
20478,O18209,560,160,?,3P1A,?,?,?,?,?,?,?,?,?,A,164,V,0.449275,0.404692,1.7,Core,?,?,?


In [202]:
struct[struct['Input-ID'].isin(['O18209'])]

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
116327,O18209,163,5VCW,A,167,H,0.446809,0.414706,2.25,Surface,NaN,NaN,O18209
116328,O18209,245,5VCW,A,250,G,0.446809,0.414706,2.25,Core,NaN,NaN,O18209
116329,O18209,103,5VCW,A,105,F,0.446809,0.414706,2.25,Core,NaN,NaN,O18209
116330,O18209,160,5VCW,A,164,V,0.446809,0.414706,2.25,Core,NaN,NaN,O18209
116331,O18209,163,5VCX,A,167,H,0.439716,0.413490,2.70,Surface,NaN,NaN,O18209
116332,O18209,245,5VCX,A,250,G,0.439716,0.413490,2.70,Core,NaN,NaN,O18209
116333,O18209,103,5VCX,A,105,F,0.439716,0.413490,2.70,Core,NaN,NaN,O18209
116334,O18209,160,5VCX,A,164,V,0.439716,0.413490,2.70,Core,NaN,NaN,O18209
116335,O18209,163,5VCV,A,167,H,0.442105,0.419118,1.92,Surface,NaN,NaN,O18209
116336,O18209,245,5VCV,A,250,G,0.442105,0.419118,1.92,ligand interaction,NaN,A:404,O18209


## resolve inconsistences

In [203]:
df = pd.read_csv('../protein_positions_with_max__rec0_pdb.tsv',sep='\t')

In [204]:
df = df.rename(columns={'PDB-ID':'PDB_ID'})

In [205]:
def resolve_inconsistence(Uniprot_AC, Pos_A, Pos_C, PDB_ID):
    if PDB_ID != 'inconsistence':
        return PDB_ID
    
    else:
        local_A = struct[(struct['Input-ID'] == Uniprot_AC)
                     & (struct['Position'] == Pos_A)]
        local_C = struct[(struct['Input-ID'] == Uniprot_AC)
                         & (struct['Position'] == Pos_C)]
        pdb_A = local_A['PDB-ID'].values.tolist()
        pdb_C = local_C['PDB-ID'].values.tolist()
        joined = [elem for elem in pdb_A if elem in pdb_C]
        
        local_A = local_A[local_A['PDB-ID'].isin(joined)]
        local_C = local_C[local_C['PDB-ID'].isin(joined)]
        local_joined = pd.merge(
            local_A,
            local_C,
            on=['PDB-ID', 'Residue RIN simple classification'],
            how='inner')
        local_joined = local_joined[(local_joined['Sequence identity_x'] > 0.7)
                                    & (local_joined['Resolution_x'] <= 2.5)]
        
        
        local_joined = local_joined.sort_values(
            by=['Sequence identity_x', 'Coverage_x'], ascending=False)

        high_priority = [
            'Protein interaction',
            'ligand interaction',
            'Peptide interaction',
            'DNA interaction',
            'metal interaction',
            'RNA interaction',
            'ion interaction',
        ]

        local_joined1 = local_joined[local_joined[
            'Residue RIN simple classification'].isin(high_priority)].copy()

        if len(local_joined1['Residue RIN simple classification'].unique()
               ) > 1:
            
            local_joined1 = local_joined1.sort_values(
                by=['Residue RIN simple classification',
                    'Sequence identity_x'], ascending=False)
            local_joined1 = local_joined1.drop_duplicates(subset='Residue RIN simple classification',
                                                         keep = 'first')
            
            
            return local_joined1['PDB-ID'].values.tolist()
        
        else:
            local_joined1 = local_joined[local_joined[
            'Residue RIN simple classification']=='Core'].copy()
            local_joined1 = local_joined1.sort_values(
                by=['Sequence identity_x', 'Coverage_x'], ascending=False)
            
            if len(local_joined1) > 0:
                return local_joined1['PDB-ID'].iloc[0]
            else:
                return local_joined['PDB-ID'].iloc[0]
                
        
            



In [206]:
df['PDB_ID'] = df.apply(
        lambda x: resolve_inconsistence(x.Uniprot_AC, x.Pos_A,x.Pos_C,x.PDB_ID),axis=1)

In [207]:
indexes = []
for i in range(len(df)):
    if pd.api.types.is_list_like(df.loc[i,'PDB_ID']):
        indexes.append(i)
       
        

In [208]:
df.loc[0]['PDB_ID']

'3UIP'

In [209]:
df.columns

Index(['Uniprot_AC', 'Pos_A', 'Pos_C', 'Max_PDB_ID_A', 'Max_PDB_ID_C',
       'Max_Chain_A', 'Max_Residue-ID_A', 'Max_PDB_AA_A',
       'Max_Sequence_identity_A', 'Max_Coverage_A', 'Max_Resolution_A',
       'Max_RIN_A', 'Max_Chain_partners_A', 'Max_Small_molecules_partners_A',
       'Max_Chain_C', 'Max_Residue-ID_C', 'Max_PDB_AA_C',
       'Max_Sequence_identity_C', 'Max_Coverage_C', 'Max_Resolution_C',
       'Max_RIN_C', 'Max_Chain_partners_C', 'Max_Small_molecules_partners_C',
       'PDB_ID'],
      dtype='object')

In [210]:
df_multiple = pd.DataFrame(columns =['Uniprot_AC', 'Pos_A', 'Pos_C', 'Max_PDB_ID_A', 'Max_PDB_ID_C',
       'Max_Chain_A', 'Max_Residue-ID_A', 'Max_PDB_AA_A',
       'Max_Sequence_identity_A', 'Max_Coverage_A', 'Max_Resolution_A',
       'Max_RIN_A', 'Max_Chain_partners_A', 'Max_Small_molecules_partners_A',
       'Max_Chain_C', 'Max_Residue-ID_C', 'Max_PDB_AA_C',
       'Max_Sequence_identity_C', 'Max_Coverage_C', 'Max_Resolution_C',
       'Max_RIN_C', 'Max_Chain_partners_C', 'Max_Small_molecules_partners_C',
       'PDB_ID'] )
for i in range(len(indexes)):
    pdbs = df.loc[indexes[0],'PDB_ID' ]
    row = df.loc[indexes[0]]
   
                   
    for elem in pdbs:
        df_multiple.loc[len(df_multiple)] = row
        df_multiple.loc[len(df_multiple)-1, 'PDB_ID'] = elem
        
        
df_multiple

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_partners_A,Max_Small_molecules_partners_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Max_Chain_partners_C,Max_Small_molecules_partners_C,PDB_ID
0,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,5F6V
1,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,3UIP
2,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,5F6V
3,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,3UIP
4,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,5F6V
5,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,3UIP
6,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,5F6V
7,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,3UIP
8,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,5F6V
9,P63279,25,42,1A3S,1A3S,A,25,V,1.0,1.0,2.8,Surface,?,?,A,42,E,1.0,1.0,2.8,Surface,?,?,3UIP


In [211]:
df.drop(indexes, axis=0, inplace=True)
df = df.append(df_multiple)

In [212]:
df = df.fillna('?')

In [213]:
df[(df['Max_PDB_ID_A']=='?')&(df['PDB_ID']!='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_partners_A,Max_Small_molecules_partners_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Max_Chain_partners_C,Max_Small_molecules_partners_C,PDB_ID


In [214]:
df[(df['Max_PDB_ID_C']=='?')&(df['PDB_ID']!='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_partners_A,Max_Small_molecules_partners_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Max_Chain_partners_C,Max_Small_molecules_partners_C,PDB_ID


## find single positions

In [215]:
struct['Position'] = struct['Position'].astype(int)
df['Pos_A'] = df['Pos_A'].astype(int)
df['Pos_C'] = df['Pos_C'].astype(int)

In [216]:
def find_pdb_single(Uniprot_AC, Pos, PDB_ID):
    if len(PDB_ID) == 4:
        return PDB_ID
    
    else:
        local_A = struct[(struct['Input-ID']==Uniprot_AC)&(struct[
                  'Position']==Pos)]
        
        local_A = local_A[(local_A['Sequence identity'] > 0.7)
                               & 
                                (local_A['Resolution'] <= 2.5)]
        
        local_A = local_A.sort_values(
                by=['Sequence identity', 'Coverage'], ascending=False)
        
        if len(local_A) == 0:
            return '?'
        
        
        else:
            
            high_priority = [
                'Protein interaction',
                'ligand interaction',
                'Peptide interaction',
                'DNA interaction',
                'metal interaction',
                'RNA interaction',
                'ion interaction',
            ]


            local_A1 = local_A[local_A[
                'Residue RIN simple classification'].isin(high_priority)]

            if len(local_A1) > 0:
                local_A1 = local_A1.sort_values(
                    by=['Sequence identity', 'Coverage'], ascending=False)
                return local_A1['PDB-ID'].iloc[0]

            else:
                local_A1 = local_A[local_A[
                'Residue RIN simple classification']=='Core'].copy()
                local_A1 = local_A1.sort_values(
                    by=['Sequence identity', 'Coverage'], ascending=False)

                if len(local_A1) > 0:
                    return local_A1['PDB-ID'].iloc[0]
                else:
                    return local_A['PDB-ID'].iloc[0]



In [217]:
df['Rec_PDB_ID_A'] = df.apply(
        lambda x: find_pdb_single(x.Uniprot_AC, x.Pos_A,x['PDB_ID']),axis=1)

In [218]:
df['Rec_PDB_ID_C'] = df.apply(
        lambda x: find_pdb_single(x.Uniprot_AC, x.Pos_C,x['PDB_ID']),axis=1)

In [219]:
df.to_csv('../protein_positions_with_max_rec1_pdb.tsv',sep='\t', index=False)

In [220]:
df[(df['Max_PDB_ID_A']=='?')&(df['Rec_PDB_ID_A']!='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_partners_A,Max_Small_molecules_partners_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Max_Chain_partners_C,Max_Small_molecules_partners_C,PDB_ID,Rec_PDB_ID_A,Rec_PDB_ID_C


In [221]:
df[(df['Max_PDB_ID_C']=='?')&(df['Rec_PDB_ID_C']!='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_partners_A,Max_Small_molecules_partners_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Max_Chain_partners_C,Max_Small_molecules_partners_C,PDB_ID,Rec_PDB_ID_A,Rec_PDB_ID_C


In [222]:
df[(df['Max_PDB_ID_A']!='?')&(df['Rec_PDB_ID_A']=='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_partners_A,Max_Small_molecules_partners_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Max_Chain_partners_C,Max_Small_molecules_partners_C,PDB_ID,Rec_PDB_ID_A,Rec_PDB_ID_C
1820,P38398,221,175,7JZV,?,A,342,T,0.630542,0.106116,3.9,Core,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1845,P38398,224,164,7JZV,?,A,345,Y,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1866,P38398,139,253,7JZV,?,A,211,S,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1870,P38398,157,253,7JZV,?,A,236,T,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1876,P38398,161,254,7JZV,?,A,253,T,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23504,A910307,97,92,1OSN,1OSN,A,97,H,0.993865,0.9560120000000001,3.2,Core,?,?,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,B,?,bad_struct,?,?
23505,A910307,97,92,1OSN,1OSN,A,97,H,0.993865,0.9560120000000001,3.2,Core,?,?,B,92,L,0.9936709999999999,0.926686,3.2,Protein interaction,A,?,bad_struct,?,?
23506,A910307,97,92,1OSN,1OSN,B,97,H,0.9936709999999999,0.926686,3.2,Core,?,?,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,B,?,bad_struct,?,?
23507,A910307,97,92,1OSN,1OSN,B,97,H,0.9936709999999999,0.926686,3.2,Core,?,?,B,92,L,0.9936709999999999,0.926686,3.2,Protein interaction,A,?,bad_struct,?,?


In [223]:
df[(df['Max_PDB_ID_C']!='?')&(df['Rec_PDB_ID_C']=='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_partners_A,Max_Small_molecules_partners_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Max_Chain_partners_C,Max_Small_molecules_partners_C,PDB_ID,Rec_PDB_ID_A,Rec_PDB_ID_C
1797,P38398,125,134,?,7JZV,?,?,?,?,?,?,?,?,?,A,206,I,0.630542,0.106116,3.9,Core,?,?,?,?,?
1803,P38398,115,134,?,7JZV,?,?,?,?,?,?,?,?,?,A,206,I,0.630542,0.106116,3.9,Core,?,?,?,?,?
1812,P38398,4,194,1JM7_AU,7JZV,A,4,S,1.0,0.055287199999999995,2.5,Surface,?,?,A,302,V,0.630542,0.106116,3.9,Core,?,?,?,1JM7_AU,?
1813,P38398,126,194,?,7JZV,?,?,?,?,?,?,?,?,?,A,302,V,0.630542,0.106116,3.9,Core,?,?,?,?,?
1833,P38398,110,224,?,7JZV,?,?,?,?,?,?,?,?,?,A,345,Y,0.630542,0.106116,3.9,Surface,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23492,A020541,274,260,6WCD,6WCD,A,270,T,0.636943,0.606178,1.54,Surface,?,?,A,256,C,0.636943,0.606178,1.54,Surface,?,?,bad_struct,?,?
23504,A910307,97,92,1OSN,1OSN,A,97,H,0.993865,0.9560120000000001,3.2,Core,?,?,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,B,?,bad_struct,?,?
23505,A910307,97,92,1OSN,1OSN,A,97,H,0.993865,0.9560120000000001,3.2,Core,?,?,B,92,L,0.9936709999999999,0.926686,3.2,Protein interaction,A,?,bad_struct,?,?
23506,A910307,97,92,1OSN,1OSN,B,97,H,0.9936709999999999,0.926686,3.2,Core,?,?,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,B,?,bad_struct,?,?


## merge with structman info

In [224]:
df = pd.read_csv('../protein_positions_with_max_rec1_pdb.tsv',sep='\t')

In [225]:
df = pd.merge(df, struct, left_on=['Uniprot_AC', 'Pos_A', 'Rec_PDB_ID_A'],
        right_on = ['Input-ID', 'Position','PDB-ID'], how='left')

In [226]:
df.drop([ 'Input-ID', 'Position', 'PDB-ID'], axis=1, inplace=True)

df = df.rename(columns={'Chain':'Chain_A', 'Residue-ID':'Residue-ID_A', 
                        'Residue AA type':'PDB_AA_A',
       'Sequence identity':'Sequence_identity_A', 
                        'Coverage':'Coverage_A', 'Resolution':'Resolution_A',
       'Residue RIN simple classification':'RIN_A', 
        'Chain type interaction partners':'Chain_partners_A',
       'Small molecules type interaction partners':
                        'Small_molecules_partners_A'})

In [227]:
df = pd.merge(df, struct, left_on=['Uniprot_AC', 'Pos_C', 'Rec_PDB_ID_C'],
        right_on = ['Input-ID', 'Position','PDB-ID'], how='left')



df.drop([ 'Input-ID', 'Position', 'PDB-ID'], axis=1, inplace=True)

df = df.rename(columns={'Chain':'Chain_C', 'Residue-ID':'Residue-ID_C', 
                        'Residue AA type':'PDB_AA_C',
       'Sequence identity':'Sequence_identity_C', 
                        'Coverage':'Coverage_C', 'Resolution':'Resolution_C',
       'Residue RIN simple classification':'RIN_C', 
        'Chain type interaction partners':'Chain_partners_C',
       'Small molecules type interaction partners':
                        'Small_molecules_partners_C'})

In [228]:
df.columns

Index(['Uniprot_AC', 'Pos_A', 'Pos_C', 'Max_PDB_ID_A', 'Max_PDB_ID_C',
       'Max_Chain_A', 'Max_Residue-ID_A', 'Max_PDB_AA_A',
       'Max_Sequence_identity_A', 'Max_Coverage_A', 'Max_Resolution_A',
       'Max_RIN_A', 'Max_Chain_partners_A', 'Max_Small_molecules_partners_A',
       'Max_Chain_C', 'Max_Residue-ID_C', 'Max_PDB_AA_C',
       'Max_Sequence_identity_C', 'Max_Coverage_C', 'Max_Resolution_C',
       'Max_RIN_C', 'Max_Chain_partners_C', 'Max_Small_molecules_partners_C',
       'PDB_ID', 'Rec_PDB_ID_A', 'Rec_PDB_ID_C', 'Protein-ID_x', 'Chain_A',
       'Residue-ID_A', 'PDB_AA_A', 'Sequence_identity_A', 'Coverage_A',
       'Resolution_A', 'RIN_A', 'Chain_partners_A',
       'Small_molecules_partners_A', 'Protein-ID_y', 'Chain_C', 'Residue-ID_C',
       'PDB_AA_C', 'Sequence_identity_C', 'Coverage_C', 'Resolution_C',
       'RIN_C', 'Chain_partners_C', 'Small_molecules_partners_C'],
      dtype='object')

In [245]:
for elem in ['Chain_A',
       'Residue-ID_A', 'PDB_AA_A', 'Sequence_identity_A', 'Coverage_A',
       'Resolution_A', 'RIN_A', 'Chain_partners_A',
       'Small_molecules_partners_A', 'Chain_C', 'Residue-ID_C', 'PDB_AA_C',
       'Sequence_identity_C', 'Coverage_C', 'Resolution_C', 'RIN_C',
       'Chain_partners_C', 'Small_molecules_partners_C']:
    
    df = df.rename(columns={elem:'Rec_'+elem})
df.head()
    

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C
0,P63279,106,74,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,74,K,1.0,1.0,2.8,Surface,3uip,3uip,A,106,A,1,0.993671,2.29,Protein interaction,D,?,A,74,K,1,0.993671,2.29,Protein interaction,C,?
1,P63279,106,67,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,67,D,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1,1,2.5,Surface,?,?,A,67,D,1,1,2.5,Surface,?,?
2,P63279,106,78,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,78,E,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1,1,2.5,Surface,?,?,A,78,E,1,1,2.5,Surface,?,?
3,P63279,106,40,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,40,N,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1,1,2.5,Surface,?,?,A,40,N,1,1,2.5,Surface,?,?
4,P63279,26,106,1a3s,1a3s,A,26,A,1.0,1.0,2.8,Core,A,106,A,1.0,1.0,2.8,Surface,4w5v,3uip,A,26,A,1,1,2.5,Core,?,?,A,106,A,1,0.993671,2.29,Protein interaction,D,?


In [230]:
df.columns

Index(['Uniprot_AC', 'Pos_A', 'Pos_C', 'Max_PDB_ID_A', 'Max_PDB_ID_C',
       'Max_Chain_A', 'Max_Residue-ID_A', 'Max_PDB_AA_A',
       'Max_Sequence_identity_A', 'Max_Coverage_A', 'Max_Resolution_A',
       'Max_RIN_A', 'Max_Chain_partners_A', 'Max_Small_molecules_partners_A',
       'Max_Chain_C', 'Max_Residue-ID_C', 'Max_PDB_AA_C',
       'Max_Sequence_identity_C', 'Max_Coverage_C', 'Max_Resolution_C',
       'Max_RIN_C', 'Max_Chain_partners_C', 'Max_Small_molecules_partners_C',
       'PDB_ID', 'Rec_PDB_ID_A', 'Rec_PDB_ID_C', 'Protein-ID_x', 'Rec_Chain_A',
       'Rec_Residue-ID_A', 'Rec_PDB_AA_A', 'Rec_Sequence_identity_A',
       'Rec_Coverage_A', 'Rec_Resolution_A', 'Rec_RIN_A',
       'Rec_Chain_partners_A', 'Rec_Small_molecules_partners_A',
       'Protein-ID_y', 'Rec_Chain_C', 'Rec_Residue-ID_C', 'Rec_PDB_AA_C',
       'Rec_Sequence_identity_C', 'Rec_Coverage_C', 'Rec_Resolution_C',
       'Rec_RIN_C', 'Rec_Chain_partners_C', 'Rec_Small_molecules_partners_C'],
      dtype='ob

In [231]:
df.drop([ 'Max_Chain_partners_A', 'Max_Small_molecules_partners_A',
        'Max_Chain_partners_C', 'Max_Small_molecules_partners_C',
        'PDB_ID','Protein-ID_x', 'Protein-ID_y',], axis=1, inplace=True)

In [232]:
# удалить колонки
# убрать в старых вариантах peptide interactions на  protein

In [233]:
df['Max_RIN_A'] = df['Max_RIN_A'].str.replace('Peptide interaction', 'Protein interaction')
df['Max_RIN_A'].value_counts()

Surface                18601
Core                    6930
Protein interaction     3913
?                       1684
ligand interaction       425
DNA interaction           84
metal interaction         19
RNA interaction           12
ion interaction            4
Name: Max_RIN_A, dtype: int64

In [234]:
df['Max_RIN_C'] = df['Max_RIN_C'].str.replace('Peptide interaction', 'Protein interaction')
df['Max_RIN_C'].value_counts()

Surface                20216
Core                    6350
Protein interaction     2772
?                       1910
ligand interaction       325
DNA interaction           70
metal interaction         20
RNA interaction            8
ion interaction            1
Name: Max_RIN_C, dtype: int64

In [235]:
df['Rec_RIN_C'] = df['Rec_RIN_C'].str.replace('1', 'Protein interaction')
df['Rec_RIN_C'].value_counts()

Surface                14175
Protein interaction     6992
Core                    4733
ligand interaction      1096
DNA interaction          194
metal interaction         20
ion interaction           16
RNA interaction            3
Name: Rec_RIN_C, dtype: int64

In [236]:
df['Rec_RIN_A'] = df['Rec_RIN_A'].str.replace('1', 'Protein interaction')
df['Rec_RIN_A'].value_counts()

Surface                13158
Protein interaction     7448
Core                    5400
ligand interaction      1213
DNA interaction          203
ion interaction           25
metal interaction         16
RNA interaction            3
Name: Rec_RIN_A, dtype: int64

In [237]:
# выкинуть _ и привести к нижнему регистру в пдб

In [238]:
for elem in ['Max_PDB_ID_A', 'Max_PDB_ID_C', 'Rec_PDB_ID_A', 'Rec_PDB_ID_C']:
    df[elem] = df[elem].str.split('_',1).str[0]
    df[elem] = df[elem].str.lower()

## проверка RIN

In [239]:
df = df.fillna('?')

In [240]:
df[(df['Max_RIN_A']=='?')&(df['Rec_RIN_A']!='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C


In [241]:
df[(df['Max_RIN_C']=='?')&(df['Rec_RIN_C']!='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C


In [242]:
df[(df['Max_RIN_A']!='?')&(df['Rec_RIN_A']=='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C
1819,P38398,221,175,7jzv,?,A,342,T,0.630542,0.106116,3.9,Core,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1844,P38398,224,164,7jzv,?,A,345,Y,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1865,P38398,139,253,7jzv,?,A,211,S,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1869,P38398,157,253,7jzv,?,A,236,T,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1875,P38398,161,254,7jzv,?,A,253,T,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31004,A910307,97,92,1osn,1osn,A,97,H,0.993865,0.9560120000000001,3.2,Core,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
31005,A910307,97,92,1osn,1osn,A,97,H,0.993865,0.9560120000000001,3.2,Core,B,92,L,0.9936709999999999,0.926686,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
31006,A910307,97,92,1osn,1osn,B,97,H,0.9936709999999999,0.926686,3.2,Core,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
31007,A910307,97,92,1osn,1osn,B,97,H,0.9936709999999999,0.926686,3.2,Core,B,92,L,0.9936709999999999,0.926686,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?


In [243]:
df[(df['Max_RIN_C']!='?')&(df['Rec_RIN_C']=='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C
1796,P38398,125,134,?,7jzv,?,?,?,?,?,?,?,A,206,I,0.630542,0.106116,3.9,Core,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1802,P38398,115,134,?,7jzv,?,?,?,?,?,?,?,A,206,I,0.630542,0.106116,3.9,Core,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1811,P38398,4,194,1jm7,7jzv,A,4,S,1.0,0.055287199999999995,2.5,Surface,A,302,V,0.630542,0.106116,3.9,Core,1jm7,?,A,4,S,1,0.0552872,2.5,Surface,?,?,?,?,?,?,?,?,?,?,?
1812,P38398,126,194,?,7jzv,?,?,?,?,?,?,?,A,302,V,0.630542,0.106116,3.9,Core,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1832,P38398,110,224,?,7jzv,?,?,?,?,?,?,?,A,345,Y,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30986,A020541,274,260,6wcd,6wcd,A,270,T,0.636943,0.606178,1.54,Surface,A,256,C,0.636943,0.606178,1.54,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
31004,A910307,97,92,1osn,1osn,A,97,H,0.993865,0.9560120000000001,3.2,Core,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
31005,A910307,97,92,1osn,1osn,A,97,H,0.993865,0.9560120000000001,3.2,Core,B,92,L,0.9936709999999999,0.926686,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
31006,A910307,97,92,1osn,1osn,B,97,H,0.9936709999999999,0.926686,3.2,Core,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?


In [244]:
df[(df['Max_RIN_A'].isin(['Surface', 'Core']))&(df['Rec_RIN_A']=='?')]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C
1819,P38398,221,175,7jzv,?,A,342,T,0.630542,0.106116,3.9,Core,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1844,P38398,224,164,7jzv,?,A,345,Y,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1865,P38398,139,253,7jzv,?,A,211,S,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1869,P38398,157,253,7jzv,?,A,236,T,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1875,P38398,161,254,7jzv,?,A,253,T,0.630542,0.106116,3.9,Surface,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31004,A910307,97,92,1osn,1osn,A,97,H,0.993865,0.9560120000000001,3.2,Core,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
31005,A910307,97,92,1osn,1osn,A,97,H,0.993865,0.9560120000000001,3.2,Core,B,92,L,0.9936709999999999,0.926686,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
31006,A910307,97,92,1osn,1osn,B,97,H,0.9936709999999999,0.926686,3.2,Core,A,92,L,0.993865,0.9560120000000001,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
31007,A910307,97,92,1osn,1osn,B,97,H,0.9936709999999999,0.926686,3.2,Core,B,92,L,0.9936709999999999,0.926686,3.2,Protein interaction,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?


In [246]:
df[['Max_RIN_A', 'Rec_RIN_A']].groupby(['Max_RIN_A', 'Rec_RIN_A']).size()

Max_RIN_A            Rec_RIN_A          
?                    ?                       1684
Core                 ?                        805
                     Core                    4653
                     DNA interaction            9
                     Protein interaction      626
                     RNA interaction            1
                     Surface                  498
                     ion interaction            1
                     ligand interaction       334
                     metal interaction          3
DNA interaction      ?                          3
                     Core                       1
                     DNA interaction           43
                     Surface                   37
Protein interaction  ?                         44
                     Core                      47
                     Protein interaction     3502
                     Surface                  311
                     ion interaction            1
         

In [247]:
df[['Max_RIN_C', 'Rec_RIN_C']].groupby(['Max_RIN_C', 'Rec_RIN_C']).size()

Max_RIN_C            Rec_RIN_C          
?                    ?                       1910
Core                 ?                        781
                     Core                    3894
                     DNA interaction           17
                     Protein interaction      860
                     RNA interaction            2
                     Surface                  418
                     ligand interaction       378
DNA interaction      ?                          5
                     DNA interaction           59
                     Surface                    6
Protein interaction  ?                         45
                     Core                      30
                     DNA interaction            4
                     Protein interaction     2372
                     Surface                  308
                     ligand interaction        13
RNA interaction      Core                       2
                     Surface                    5
         

In [248]:
df.to_csv('../protein_positions_with_max_rec2_pdb.tsv',sep='\t', index=False)

# merge with mutations scores

In [249]:
df = pd.read_csv('../../CPD_data_distance_26_03_21.tsv.gz',sep='\t')

/Users/azbukinanadezda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9,10,11,18,20,21,22,27,29,30,31,38,40,41,42,50,52,53,54) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [250]:
df.columns

Index(['#Pair_Type', 'Dataset', 'Uniprot_AC', 'Pos_A', 'A1', 'A2', 'Pos_C',
       'C1', 'C2', 'Score_A', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC', 'Max_PDB_ID_A', 'Max_Chain_A',
       'Max_Residue-ID_A', 'Max_PDB_AA_A', 'Max_Sequence_identity_A',
       'Max_Coverage_A', 'Max_Resolution_A', 'Short_Max_RIN_A',
       'Max_Iden_A1 match', 'Max_PDB_ID_C', 'Max_Chain_C', 'Max_Residue-ID_C',
       'Max_PDB_AA_C', 'Max_Sequence_identity_C', 'Max_Coverage_C',
       'Max_Resolution_C', 'Short_Max_RIN_C', 'Max_Iden_C1_match',
       'Max_Dist_ca', 'Max_Dist_closest', 'Rec_PDB_ID_A', 'Rec_Chain_A',
       'Rec_Residue-ID_A', 'Rec_PDB_AA_A', 'Rec_Sequence_identity_A',
       'Rec_Coverage_A', 'Rec_Resolution_A', 'Rec_RIN_A', 'Rec_Short_RIN_A',
       'Rec_Chain_partners_A', 'Rec_Small_molecules_partners_A',
       'Recom_A1_match', 'Rec_PDB_ID_C', 'Rec_Chain_C', 'Rec_Residue-ID_C',
       'Rec_PDB_AA_C', 'Rec_Sequence_identity_C', 'Rec_Coverage_C',
       

In [251]:
df = df[['#Pair_Type', 'Dataset', 'Uniprot_AC', 'Pos_A', 'A1', 'A2', 'Pos_C',
       'C1', 'C2', 'Score_A', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC']]
df = df.drop_duplicates()
len(df)

671711

In [252]:
df['Pos_A'] = df['Pos_A'].astype(int)
df['Pos_C'] = df['Pos_C'].astype(int)



In [253]:
df.loc[df['Uniprot_AC']=='BLAT_ECOLX', 'Uniprot_AC'] = 'P62593'
df.loc[df['Uniprot_AC']=='MCP2_ECOLI', 'Uniprot_AC'] = 'P07017'
df.loc[df['Uniprot_AC']=='P53_HUMAN', 'Uniprot_AC'] = 'P04637'
df.loc[df['Uniprot_AC']=='PMY13_CAEEL', 'Uniprot_AC'] = 'O18209'

df['Uniprot_AC_2'] = df['Uniprot_AC'].str.split(':',1).str[0]

In [254]:
dfstruc = pd.read_csv('../protein_positions_with_max_rec2_pdb.tsv',sep='\t')
dfstruc

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C
0,P63279,106,74,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,74,K,1.0,1.0,2.8,Surface,3uip,3uip,A,106,A,1.0,0.9936709999999999,2.29,Protein interaction,D,?,A,74,K,1.0,0.9936709999999999,2.29,Protein interaction,C,?
1,P63279,106,67,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,67,D,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1.0,1.0,2.5,Surface,?,?,A,67,D,1.0,1.0,2.5,Surface,?,?
2,P63279,106,78,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,78,E,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1.0,1.0,2.5,Surface,?,?,A,78,E,1.0,1.0,2.5,Surface,?,?
3,P63279,106,40,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,40,N,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1.0,1.0,2.5,Surface,?,?,A,40,N,1.0,1.0,2.5,Surface,?,?
4,P63279,26,106,1a3s,1a3s,A,26,A,1.0,1.0,2.8,Core,A,106,A,1.0,1.0,2.8,Surface,4w5v,3uip,A,26,A,1.0,1.0,2.5,Core,?,?,A,106,A,1.0,0.9936709999999999,2.29,Protein interaction,D,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31667,P63279,25,42,1a3s,1a3s,A,25,V,1.0,1.0,2.8,Surface,A,42,E,1.0,1.0,2.8,Surface,3uip,3uip,A,25,V,1.0,0.9936709999999999,2.29,Protein interaction,D,?,A,42,E,1.0,0.9936709999999999,2.29,Protein interaction,D,?
31668,P63279,25,42,1a3s,1a3s,A,25,V,1.0,1.0,2.8,Surface,A,42,E,1.0,1.0,2.8,Surface,5f6v,5f6v,A,25,V,0.980769,0.987342,1.49,ligand interaction,?,A:201,A,42,E,0.980769,0.987342,1.49,ligand interaction,?,A:201
31669,P63279,25,42,1a3s,1a3s,A,25,V,1.0,1.0,2.8,Surface,A,42,E,1.0,1.0,2.8,Surface,3uip,3uip,A,25,V,1.0,0.9936709999999999,2.29,Protein interaction,D,?,A,42,E,1.0,0.9936709999999999,2.29,Protein interaction,D,?
31670,P63279,25,42,1a3s,1a3s,A,25,V,1.0,1.0,2.8,Surface,A,42,E,1.0,1.0,2.8,Surface,5f6v,5f6v,A,25,V,0.980769,0.987342,1.49,ligand interaction,?,A:201,A,42,E,0.980769,0.987342,1.49,ligand interaction,?,A:201


In [255]:
#pd.merge(df,dfstruc, )

In [256]:
df_joined = pd.merge(df, dfstruc, how='left', 
                     left_on=['Uniprot_AC_2','Pos_A', 'Pos_C' ],
                    right_on=['Uniprot_AC','Pos_A', 'Pos_C' ])

In [257]:
len(df_joined)

773473

In [258]:
len(df_joined.drop_duplicates(subset=['#Pair_Type', 'Dataset', 'Uniprot_AC_x', 'Pos_A', 'A1', 'A2', 'Pos_C',
       'C1', 'C2', 'Score_A', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC']))

671711

In [259]:
df_joined[df_joined['Max_PDB_ID_A']=='?']['Dataset'].value_counts()

maveDB_15    3896
maveDB_13    1538
maveDB_8      820
maveDB_6      481
PMD_PMD       328
PMD_Poon      101
Name: Dataset, dtype: int64

In [260]:
set(df_joined[(df_joined['Max_PDB_ID_A']=='?')&(df_joined[
    'Dataset']=='PMD_Poon')]['Uniprot_AC_x'].values.tolist())

{'LACY_STRTR',
 'MELB_ECOLI',
 'O18209',
 'P07017',
 'TCR3_ECOLX',
 'VG05_BPP22',
 'VGA_BPPHX',
 'VGC_BPPHX',
 'VGF_BPPHX',
 'VGG_BPPHX',
 'VGH_BPPHX'}

In [261]:
df_joined[df_joined['Max_PDB_ID_C']=='?']['Dataset'].value_counts()

maveDB_15    15588
maveDB_13     1678
maveDB_8       939
maveDB_6       601
PMD_PMD        322
PMD_Poon        95
Name: Dataset, dtype: int64

In [262]:
set(df_joined[(df_joined['Max_PDB_ID_C']=='?')&(df_joined[
    'Dataset']=='PMD_Poon')]['Uniprot_AC_x'].values.tolist())

{'LACY_STRTR',
 'MELB_ECOLI',
 'STE6_YEAST',
 'TCR3_ECOLX',
 'VG05_BPP22',
 'VGA_BPPHX',
 'VGC_BPPHX',
 'VGF_BPPHX',
 'VGG_BPPHX',
 'VGH_BPPHX'}

In [264]:
df_joined.drop([ 'Uniprot_AC_2', 'Uniprot_AC_y'], axis=1, inplace=True)
df_joined = df_joined.rename(columns={'Uniprot_AC_x':'Uniprot_AC'})

In [265]:
df_joined

,#Pair_Type,Dataset,Uniprot_AC,Pos_A,A1,A2,Pos_C,C1,C2,Score_A,Score_C,Score_AC,PMD_Phen,Effect_A,Effect_C,Effect_AC,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C
0,no_type,maveDB_2,P63279,106,A,E,74,K,D,-3.15209,0.421136,-4.32193,?,bad,neutral,bad,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,74,K,1.0,1.0,2.8,Surface,3uip,3uip,A,106,A,1.0,0.9936709999999999,2.29,Protein interaction,D,?,A,74,K,1.0,0.9936709999999999,2.29,Protein interaction,C,?
1,no_type,maveDB_2,P63279,106,A,I,67,D,T,-1.46562,-0.649785,-2.00309,?,bad,neutral,bad,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,67,D,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1.0,1.0,2.5,Surface,?,?,A,67,D,1.0,1.0,2.5,Surface,?,?
2,no_type,maveDB_2,P63279,106,A,T,78,E,G,-0.200297,-0.0721246,0.71154,?,neutral,neutral,neutral,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,78,E,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1.0,1.0,2.5,Surface,?,?,A,78,E,1.0,1.0,2.5,Surface,?,?
3,no_type,maveDB_2,P63279,106,A,T,40,N,D,-0.200297,0.16965,-0.0213069,?,neutral,neutral,neutral,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,40,N,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1.0,1.0,2.5,Surface,?,?,A,40,N,1.0,1.0,2.5,Surface,?,?
4,no_type,maveDB_2,P63279,26,A,G,106,A,T,-1.10596,-0.200297,-0.791497,?,bad,neutral,neutral,1a3s,1a3s,A,26,A,1.0,1.0,2.8,Core,A,106,A,1.0,1.0,2.8,Surface,4w5v,3uip,A,26,A,1.0,1.0,2.5,Core,?,?,A,106,A,1.0,0.9936709999999999,2.29,Protein interaction,D,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773468,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,L,47,I,L,0,1,0,'Affinity for DNA',bad,neutral,bad,2gn5,2gn5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2gn5,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?
773469,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,L,47,I,M,0,1,0,'Affinity for DNA',bad,neutral,bad,2gn5,2gn5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2gn5,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?
773470,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,M,47,I,L,0,1,0,'Affinity for DNA',bad,neutral,bad,2gn5,2gn5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2gn5,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?
773471,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,M,47,I,M,0,1,0,'Affinity for DNA',bad,neutral,bad,2gn5,2gn5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2gn5,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?


In [266]:
df_joined.to_csv('../../CPD_data_no_distance_26_03_21_2.tsv.gz',sep='\t',index=False)



# Добить ? в residues ID

In [502]:
df = pd.read_csv('../../CPD_data_no_distance_26_03_21_2.tsv.gz',sep='\t')
for elem in df.columns:
    df = df.rename(columns={elem:elem.replace('.','_')})
df.head()

/Users/azbukinanadezda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9,10,11,19,21,22,23,26,28,29,30,35,37,38,39,44,46,47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Pair_Type,Dataset,Uniprot_AC,Pos_A,A1,A2,Pos_C,C1,C2,Score_A,Score_C,Score_AC,PMD_Phen,Effect_A,Effect_C,Effect_AC,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C
0,no_type,maveDB_2,P63279,106,A,E,74,K,D,-3.15209,0.421136,-4.32193,?,bad,neutral,bad,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,74,K,1.0,1.0,2.8,Surface,3uip,3uip,A,106,A,1.0,0.9936709999999999,2.29,Protein interaction,D,?,A,74,K,1.0,0.9936709999999999,2.29,Protein interaction,C,?
1,no_type,maveDB_2,P63279,106,A,I,67,D,T,-1.46562,-0.649785,-2.00309,?,bad,neutral,bad,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,67,D,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1.0,1.0,2.5,Surface,?,?,A,67,D,1.0,1.0,2.5,Surface,?,?
2,no_type,maveDB_2,P63279,106,A,T,78,E,G,-0.200297,-0.0721246,0.71154,?,neutral,neutral,neutral,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,78,E,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1.0,1.0,2.5,Surface,?,?,A,78,E,1.0,1.0,2.5,Surface,?,?
3,no_type,maveDB_2,P63279,106,A,T,40,N,D,-0.200297,0.16965,-0.0213069,?,neutral,neutral,neutral,1a3s,1a3s,A,106,A,1.0,1.0,2.8,Surface,A,40,N,1.0,1.0,2.8,Surface,4w5v,4w5v,A,106,A,1.0,1.0,2.5,Surface,?,?,A,40,N,1.0,1.0,2.5,Surface,?,?
4,no_type,maveDB_2,P63279,26,A,G,106,A,T,-1.10596,-0.200297,-0.791497,?,bad,neutral,neutral,1a3s,1a3s,A,26,A,1.0,1.0,2.8,Core,A,106,A,1.0,1.0,2.8,Surface,4w5v,3uip,A,26,A,1.0,1.0,2.5,Core,?,?,A,106,A,1.0,0.9936709999999999,2.29,Protein interaction,D,?


In [440]:
struct.head()

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
0,A992881:1:[DIRECT],365,4x0f,A,586,F,0.972010,0.837953,3.22,Protein interaction,B,A:804,A992881
1,A992881:1:[DIRECT],354,4x0f,A,575,Y,0.972010,0.837953,3.22,ligand interaction,NaN,A:804,A992881
2,A992881:1:[DIRECT],401,4x0f,A,622,I,0.972010,0.837953,3.22,Protein interaction,B,NaN,A992881
3,A992881:1:[DIRECT],383,4x0f,A,604,A,0.972010,0.837953,3.22,Core,NaN,NaN,A992881
4,A992881:1:[DIRECT],365,4wzi,B,586,F,0.971867,0.807851,2.58,Protein interaction,A,NaN,A992881


In [503]:
len(df)

773473

In [504]:
struct['Position'] = struct['Position'].astype(int)

In [505]:
df['Uniprot_AC_2'] = df['Uniprot_AC'].str.split(':',1).str[0]

In [506]:
df = pd.merge(df, struct, how='left',
        left_on=['Pos_A', 'Rec_PDB_ID_A','Rec_Chain_A', 'Rec_RIN_A', 'Uniprot_AC_2'],
        right_on=['Position', 'PDB-ID', 'Chain', 'Residue RIN simple classification','Input-ID'])
df['Rec_Residue-ID_A'] = df['Residue-ID']
df = df.fillna('?')
df[(df['Rec_Residue-ID_A']=='?')&(df['Rec_PDB_ID_A']!='?')].index

Int64Index([], dtype='int64')

In [507]:
df.drop(['Protein-ID', 'Position', 'PDB-ID', 'Chain',
       'Residue-ID', 'Residue AA type', 'Sequence identity', 'Coverage',
       'Resolution', 'Residue RIN simple classification',
       'Chain type interaction partners',
       'Small molecules type interaction partners', 'Input-ID',], axis=1, inplace = True)


In [508]:
len(df)

773473

In [509]:
df = pd.merge(df, struct, how='left',
        left_on=['Pos_C', 'Rec_PDB_ID_C','Rec_Chain_C', 'Rec_RIN_C', 'Uniprot_AC_2'],
        right_on=['Position', 'PDB-ID', 'Chain', 'Residue RIN simple classification','Input-ID'])
df = df.fillna('?')
df['Rec_Residue-ID_C'] = df['Residue-ID']
df[(df['Rec_Residue-ID_C']=='?')&(df['Rec_PDB_ID_C']!='?')].index

Int64Index([], dtype='int64')

In [510]:
df.drop(['Protein-ID', 'Position', 'PDB-ID', 'Chain',
       'Residue-ID', 'Residue AA type', 'Sequence identity', 'Coverage',
       'Resolution', 'Residue RIN simple classification',
       'Chain type interaction partners',
       'Small molecules type interaction partners', 'Input-ID',], axis=1, inplace = True)


In [511]:
df = pd.merge(df, struct, how='left',
        left_on=['Pos_C', 'Max_PDB_ID_C','Max_Chain_C', 'Max_RIN_C', 'Uniprot_AC_2'],
        right_on=['Position', 'PDB-ID', 'Chain', 'Residue RIN simple classification','Input-ID'])
df = df.fillna('?')
df['Max_Residue-ID_C'] = df['Residue-ID']
df[(df['Max_Residue-ID_C']=='?')&(df['Max_PDB_ID_C']!='?')].index

Int64Index([762999, 763000, 766546, 766547, 766548, 766549, 766550, 766551,
            766552, 766553, 766554, 766555, 766556, 766557, 766558, 766559,
            766560, 766561, 768850, 768851],
           dtype='int64')

In [512]:
df.drop(['Protein-ID', 'Position', 'PDB-ID', 'Chain',
       'Residue-ID', 'Residue AA type', 'Sequence identity', 'Coverage',
       'Resolution', 'Residue RIN simple classification',
       'Chain type interaction partners',
       'Small molecules type interaction partners', 'Input-ID',], axis=1, inplace = True)


In [513]:
df = pd.merge(df, struct, how='left',
        left_on=['Pos_A', 'Max_PDB_ID_A','Max_Chain_A', 'Max_RIN_A', 'Uniprot_AC_2'],
        right_on=['Position', 'PDB-ID', 'Chain', 'Residue RIN simple classification','Input-ID'])
df = df.fillna('?')
df['Max_Residue-ID_A'] = df['Residue-ID']
df[(df['Max_Residue-ID_A']=='?')&(df['Max_PDB_ID_A']!='?')].index

Int64Index([766412, 767025, 769971, 769988], dtype='int64')

In [514]:
df[(df['Max_Residue-ID_A']=='?')&(df['Max_PDB_ID_A']!='?')]

,#Pair_Type,Dataset,Uniprot_AC,Pos_A,A1,A2,Pos_C,C1,C2,Score_A,Score_C,Score_AC,PMD_Phen,Effect_A,Effect_C,Effect_AC,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C,Uniprot_AC_2,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
766412,no_type,PMD_PMD,A012101:1:FAK1_HUMAN,925,Y,F,397,Y,F,1,1,1,'Expression level',neutral,neutral,neutral,1ow8,2j0j,A,?,Y,1.0,0.127376,2.85,Protein interaction,A,397,Y,0.9689030000000001,0.580799,2.8,Surface,1k04,6cb0,A,925,Y,1.0,0.134981,1.95,ion interaction,?,A:200,A,397,Y,0.955182,0.339354,1.97,Surface,?,?,A012101,?,?,?,?,?,?,?,?,?,?,?,?,?
767025,no_type,PMD_PMD,A880354:1:HB2K_MOUSE,103,V,A,100,L,V,0,0,0,'and K24-199',bad,bad,bad,1iak,1iak,B,?,V,0.983784,0.703422,1.9,Protein interaction,B,75,L,0.983784,0.703422,1.9,Surface,6blr,6blr,B,77,A,0.857143,0.7396229999999999,1.96,Core,?,?,B,74,L,0.857143,0.7396229999999999,1.96,Core,?,?,A880354,?,?,?,?,?,?,?,?,?,?,?,?,?
769971,no_type,PMD_PMD,A921285:1:QRHUE,538,D,N,523,E,Q,1,1,1,'Inhibitory effect of antiestrogens on estroge...,neutral,neutral,neutral,1x7r,1x7r,A,?,D,1.0,0.394958,2.0,Protein interaction,A,523,E,1.0,0.394958,2.0,Surface,3ert,3ert,A,538,D,0.995951,0.415126,1.9,Surface,?,?,A,523,E,0.995951,0.415126,1.9,Surface,?,?,A921285,?,?,?,?,?,?,?,?,?,?,?,?,?
769988,no_type,PMD_PMD,A940958:1:RARA_MOUSE,412,E,Q,415,E,Q,0,1,0,'tRA-dependent activation',bad,neutral,bad,3a9e,3a9e,B,?,E,0.995816,0.517316,2.75,Protein interaction,B,415,E,0.995816,0.517316,2.75,Surface,5k13,5k13,A,412,E,0.9957450000000001,0.5086579999999999,1.85,Surface,?,?,A,415,E,0.9957450000000001,0.5086579999999999,1.85,Surface,?,?,A940958,?,?,?,?,?,?,?,?,?,?,?,?,?


In [515]:
Pos = 103
pdb = '1iak'
chain = 'B'

struct[(struct['Position']==Pos)&
       (struct['PDB-ID'].str.contains(pdb))&
       (struct['Chain']==chain)]

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
298052,A880354:1:HB2K_MOUSE,103,1iak,B,78,V,0.983784,0.703422,1.9,Peptide interaction,"P,A",NaN,A880354


In [516]:
indexes = df[(df['Max_Residue-ID_A']=='?')&(df['Max_PDB_ID_A']!='?')].index
for i in indexes:
    print(i)
    Pos = int(df.loc[i, 'Pos_A'])
    
    pdb = str(df.loc[i, 'Max_PDB_ID_A'])
    
    chain = str(df.loc[i, 'Max_Chain_A'])
    new_pos = struct[(struct['Position']==Pos)&
       (struct['PDB-ID'].str.contains(pdb))&
       (struct['Chain']==chain)]
    
    df.loc[i,'Max_Residue-ID_A'] = new_pos['Residue-ID'].iloc[0]
    



766412
767025
769971
769988


In [517]:
indexes = df[(df['Max_Residue-ID_C']=='?')&(df['Max_PDB_ID_C']!='?')].index
for i in indexes:
    Pos = int(df.loc[i, 'Pos_C'])
    pdb = str(df.loc[i, 'Max_PDB_ID_C'])
    chain = str(df.loc[i, 'Max_Chain_C'])
    new_pos = struct[(struct['Position']==Pos)&
       (struct['PDB-ID'].str.contains(pdb))&
       (struct['Chain']==chain)]
    
    df.loc[i,'Max_Residue-ID_C'] = new_pos['Residue-ID'].iloc[0]
    




In [518]:
df[(df['Max_Residue-ID_C']=='?')&(df['Max_PDB_ID_C']!='?')].index

Int64Index([], dtype='int64')

In [519]:
for elem in df.columns:
    df = df.rename(columns={elem:elem.replace('_','.')})

In [483]:
df = df[['#Pair.Type', 'Dataset', 'Uniprot.AC', 'Pos.A', 'A1', 'A2', 'Pos.C',
       'C1', 'C2', 'Score.A', 'Score.C', 'Score.AC', 'PMD.Phen', 'Effect.A',
       'Effect.C', 'Effect.AC', 'Max.PDB.ID.A', 'Max.Chain.A',
       'Max.Residue-ID.A', 'Max.PDB.AA.A', 'Max.Sequence.identity.A',
       'Max.Coverage.A', 'Max.Resolution.A', 'Max.RIN.A', 'Max.Iden.A1 match',
      'Max.PDB.ID.C', 'Max.Chain.C', 'Max.Residue-ID.C',
       'Max.PDB.AA.C', 'Max.Sequence.identity.C', 'Max.Coverage.C',
       'Max.Resolution.C', 'Max.RIN.C', 'Max.Iden.C1.match', 
       'Rec.PDB.ID.A', 'Rec.Chain.A',
       'Rec.Residue-ID.A', 'Rec.PDB.AA.A', 'Rec.Sequence.identity.A',
       'Rec.Coverage.A', 'Rec.Resolution.A', 'Rec.RIN.A',
       'Rec.Chain.partners.A', 'Rec.Small.molecules.partners.A',
       'Rec.A1.match',  'Rec.PDB.ID.C', 'Rec.Chain.C',
       'Rec.Residue-ID.C', 'Rec.PDB.AA.C', 'Rec.Sequence.identity.C',
       'Rec.Coverage.C', 'Rec.Resolution.C', 'Rec.RIN.C',
       'Rec.Chain.partners.C', 'Rec.Small.molecules.partners.C',
       'Rec.C1.match',  ]]

In [520]:
df.to_csv('../../CPD_data_no_distance_29_03_21_forVER.tsv.gz',sep='\t',index=False)

In [564]:
df1 = pd.read_csv('../../CPD_data_no_distance_29_03_21_forVER.tsv.gz',sep='\t')
len(df1)

/Users/azbukinanadezda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9,10,11,19,21,22,23,26,28,29,30,35,37,38,39,44,46,47,48,54,57,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


773473

# filter distances

In [562]:
df = pd.read_csv('../../CPD_data_distance_30_03_21.tsv.gz',sep='\t')

for elem in df.columns:
    df = df.rename(columns={elem:elem.replace('.','_')})
df.head()

/Users/azbukinanadezda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9,10,11,21,22,23,28,29,30,37,38,39,46,47,48,54,57,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Pair_Type,Dataset,Uniprot_AC,Pos_A,A1,A2,Pos_C,C1,C2,Score_A,Score_C,Score_AC,PMD_Phen,Effect_A,Effect_C,Effect_AC,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C,Uniprot_AC_2,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID,Rec_Dist_ca,Rec_Dist_closest,Max_Dist_ca,Max_Dist_closest
0,no_type,maveDB_2,P63279,106,A,E,74,K,D,-3.15209,0.421136,-4.32193,?,bad,neutral,bad,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,K,1,1,2.8,Surface,3UIP,3UIP,A,?,A,1,0.993671,2.29,Protein interaction,D,?,A,?,K,1,0.993671,2.29,Protein interaction,C,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
1,no_type,maveDB_2,P63279,106,A,I,67,D,T,-1.46562,-0.649785,-2.00309,?,bad,neutral,bad,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,D,1,1,2.8,Surface,4W5V,4W5V,A,?,A,1,1,2.5,Surface,?,?,A,?,D,1,1,2.5,Surface,?,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
2,no_type,maveDB_2,P63279,106,A,T,78,E,G,-0.200297,-0.0721246,0.71154,?,neutral,neutral,neutral,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,E,1,1,2.8,Surface,4W5V,4W5V,A,?,A,1,1,2.5,Surface,?,?,A,?,E,1,1,2.5,Surface,?,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
3,no_type,maveDB_2,P63279,106,A,T,40,N,D,-0.200297,0.16965,-0.0213069,?,neutral,neutral,neutral,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,N,1,1,2.8,Surface,4W5V,4W5V,A,?,A,1,1,2.5,Surface,?,?,A,?,N,1,1,2.5,Surface,?,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
4,no_type,maveDB_2,P63279,26,A,G,106,A,T,-1.10596,-0.200297,-0.791497,?,bad,neutral,neutral,1A3S,1A3S,A,?,A,1,1,2.8,Core,A,?,A,1,1,2.8,Surface,4W5V,3UIP,A,?,A,1,1,2.5,Core,?,?,A,?,A,1,0.993671,2.29,Protein interaction,D,?,P63279,P63279,26,1a3s,A,26,A,1,1,2.8,Core,?,?,P63279,?,?,?,?


In [565]:
len(df)

772968

In [563]:
df[(df['Rec_Residue-ID_A']=='?')&(df['Rec_PDB_ID_A']!='?')].index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            745462, 745463, 745464, 745465, 745466, 745467, 745468, 745469,
            745470, 745471],
           dtype='int64', length=229376)

In [522]:
duplicated = df[df.duplicated(subset=['#Pair_Type', 'PMD_Phen', 'Uniprot_AC',
                                      'Pos_A', 'A1', 'A2', 'Pos_C','C1', 'C2',
                                     'Rec_PDB_ID_A', 'Rec_PDB_ID_C',
                                     'Rec_RIN_A', 'Rec_RIN_C'],
                keep = False)].sort_values(
    by = ['#Pair_Type', 'Uniprot_AC','Pos_A', 'A1', 'A2']).copy()
duplicated.shape




(121620, 70)

In [523]:
duplicated_unique = duplicated.drop_duplicates(subset=['#Pair_Type', 'PMD_Phen', 
                                                'Uniprot_AC','Pos_A', 
                                                'A1', 'A2', 'Pos_C',
                                                'C1', 'C2',
                                                'Rec_PDB_ID_A', 'Rec_PDB_ID_C',
                                     'Rec_RIN_A', 'Rec_RIN_C' ],
                keep = 'first')
duplicated_unique.shape


(35089, 70)

In [524]:
keep_all = []
delete_all = []

i = 0
for index, row in duplicated_unique.iterrows():
    if i%1000 == 0:
        print(i/len(duplicated_unique))
    i += 1
    
    #choose pool of the identically mapped residues
    local = duplicated[(duplicated['Dataset'] == row['Dataset'])&
                (duplicated['Uniprot_AC'] == row['Uniprot_AC'])&(
        duplicated['Pos_A'] == row['Pos_A'])&(
        duplicated['A1'] == row['A1'])&(
        duplicated['A2'] == row['A2']) &(          
        duplicated['Pos_C'] == row['Pos_C'])&(
        duplicated['C1'] == row['C1'])&(
        duplicated['C2'] == row['C2'])&(
        duplicated['Rec_PDB_ID_A'] == row['Rec_PDB_ID_A'])&(
        duplicated['Rec_PDB_ID_C'] == row['Rec_PDB_ID_C'])&(
        duplicated['PMD_Phen'] == row['PMD_Phen'])
              &(
        duplicated['Rec_RIN_A'] == row['Rec_RIN_A']) &(
        duplicated['Rec_RIN_C'] == row['Rec_RIN_C'])]

        
    local = local.sort_values(by = 'Rec_Dist_closest',ascending = True)
    
    keep = local.index.tolist()[0]
    delete = local.index.tolist()[1:]
    
    keep_all.append(keep)
    delete_all.extend(delete)
    




0.0
0.02849895978796774
0.05699791957593548
0.08549687936390321
0.11399583915187096
0.14249479893983868
0.17099375872780642
0.1994927185157742
0.22799167830374192
0.25649063809170963
0.28498959787967737
0.3134885576676451
0.34198751745561284
0.37048647724358064
0.3989854370315484
0.4274843968195161
0.45598335660748385
0.4844823163954516
0.5129812761834193
0.5414802359713871
0.5699791957593547
0.5984781555473225
0.6269771153352902
0.655476075123258
0.6839750349112257
0.7124739946991935
0.7409729544871613
0.769471914275129
0.7979708740630967
0.8264698338510644
0.8549687936390322
0.8834677534269999
0.9119667132149677
0.9404656730029354
0.9689646327909032
0.9974635925788709


In [525]:
df = df.drop(delete_all)
df.reset_index(drop=True, inplace=True)
df.Dataset.value_counts()

maveDB_15        546708
maveDB_13         51899
maveDB_14         36511
maveDB_17         21295
maveDB_4          19360
maveDB_8           4213
PMD_PMD            2622
maveDB_6           2152
maveDB_2           1152
PMD_Poon            361
CAGI                347
Jacquier_2013       200
Name: Dataset, dtype: int64

In [280]:
#df.to_csv('../../CPD_data_distance_2_03_21_v1.tsv.gz',sep='\t',index=False)




In [530]:
len(df)

686820

## max identity

In [531]:
duplicated = df[df.duplicated(subset=['#Pair_Type', 'PMD_Phen', 'Uniprot_AC',
                                      'Pos_A', 'A1', 'A2', 'Pos_C','C1', 'C2',
                                     'Max_PDB_ID_A', 'Max_PDB_ID_C',
                                     ],
                keep = False)].sort_values(
    by = ['#Pair_Type', 'Uniprot_AC','Pos_A', 'A1', 'A2']).copy()
duplicated.shape





(29398, 70)

In [532]:
duplicated_unique = duplicated.drop_duplicates(subset=['#Pair_Type', 'PMD_Phen', 
                                                'Uniprot_AC','Pos_A', 
                                                'A1', 'A2', 'Pos_C',
                                                'C1', 'C2',
                                                'Max_PDB_ID_A', 'Max_PDB_ID_C',
                                     'Max_RIN_A', 'Max_RIN_C' ],
                keep = 'first')
duplicated_unique.shape



(13913, 70)

In [533]:
keep_all = []
delete_all = []

i = 0
for index, row in duplicated_unique.iterrows():
    if i%1000 == 0:
        print(i/len(duplicated_unique))
    i += 1
    
    #choose pool of the identically mapped residues
    local = duplicated[(duplicated['Dataset'] == row['Dataset'])&
                (duplicated['Uniprot_AC'] == row['Uniprot_AC'])&(
        duplicated['Pos_A'] == row['Pos_A'])&(
        duplicated['A1'] == row['A1'])&(
        duplicated['A2'] == row['A2']) &(          
        duplicated['Pos_C'] == row['Pos_C'])&(
        duplicated['C1'] == row['C1'])&(
        duplicated['C2'] == row['C2'])&(
        duplicated['Rec_PDB_ID_A'] == row['Rec_PDB_ID_A'])&(
        duplicated['Rec_PDB_ID_C'] == row['Rec_PDB_ID_C'])&(
        duplicated['PMD_Phen'] == row['PMD_Phen'])
              &(
        duplicated['Rec_RIN_A'] == row['Rec_RIN_A']) &(
        duplicated['Rec_RIN_C'] == row['Rec_RIN_C'])]

        
    local = local.sort_values(by = 'Rec_Dist_closest',ascending = True)
    
    keep = local.index.tolist()[0]
    delete = local.index.tolist()[1:]
    
    keep_all.append(keep)
    delete_all.extend(delete)
    





0.0
0.07187522461007691
0.14375044922015381
0.2156256738302307
0.28750089844030763
0.3593761230503845
0.4312513476604614
0.5031265722705384
0.5750017968806153
0.6468770214906922
0.718752246100769
0.7906274707108459
0.8625026953209228
0.9343779199309998


In [534]:
df = df.drop(delete_all)
df.reset_index(drop=True, inplace=True)
df.Dataset.value_counts()

maveDB_15        546708
maveDB_13         51899
maveDB_14         36511
maveDB_17         21295
maveDB_4          19360
maveDB_8           4213
PMD_PMD            2621
maveDB_6           2152
maveDB_2           1152
PMD_Poon            361
CAGI                347
Jacquier_2013       200
Name: Dataset, dtype: int64

In [535]:
len(df)

686819

In [536]:
len(df)

686819

In [537]:
df.to_csv('../../CPD_data_distance_30_03_21_2.tsv.gz',sep='\t',index=False)





In [538]:
df.drop_duplicates(subset=['#Pair_Type', 'Dataset', 'Uniprot_AC', 
                               'Pos_A', 'A1', 'A2', 'Score_A',
       'Pos_C', 'C1', 'C2', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC', 'Max_PDB_ID_A', 'Max_Chain_A',
       'Max_Residue-ID_A', 'Max_PDB_AA_A', 'Max_Sequence_identity_A',
       'Max_Coverage_A', 'Max_Resolution_A', 'Max_PDB_ID_C', 'Max_Chain_C',
       'Max_Residue-ID_C', 'Max_PDB_AA_C', 'Max_Sequence_identity_C',
       'Max_Coverage_C', 'Max_Resolution_C', 'Max_Dist_ca', 'Max_Dist_closest',
       ])

,#Pair_Type,Dataset,Uniprot_AC,Pos_A,A1,A2,Pos_C,C1,C2,Score_A,Score_C,Score_AC,PMD_Phen,Effect_A,Effect_C,Effect_AC,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C,Uniprot_AC_2,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID,Rec_Dist_ca,Rec_Dist_closest,Max_Dist_ca,Max_Dist_closest
0,no_type,maveDB_2,P63279,106,A,E,74,K,D,-3.15209,0.421136,-4.32193,?,bad,neutral,bad,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,K,1,1,2.8,Surface,3UIP,3UIP,A,?,A,1,0.993671,2.29,Protein interaction,D,?,A,?,K,1,0.993671,2.29,Protein interaction,C,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
1,no_type,maveDB_2,P63279,106,A,I,67,D,T,-1.46562,-0.649785,-2.00309,?,bad,neutral,bad,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,D,1,1,2.8,Surface,4W5V,4W5V,A,?,A,1,1,2.5,Surface,?,?,A,?,D,1,1,2.5,Surface,?,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
2,no_type,maveDB_2,P63279,106,A,T,78,E,G,-0.200297,-0.0721246,0.71154,?,neutral,neutral,neutral,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,E,1,1,2.8,Surface,4W5V,4W5V,A,?,A,1,1,2.5,Surface,?,?,A,?,E,1,1,2.5,Surface,?,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
3,no_type,maveDB_2,P63279,106,A,T,40,N,D,-0.200297,0.16965,-0.0213069,?,neutral,neutral,neutral,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,N,1,1,2.8,Surface,4W5V,4W5V,A,?,A,1,1,2.5,Surface,?,?,A,?,N,1,1,2.5,Surface,?,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
4,no_type,maveDB_2,P63279,26,A,G,106,A,T,-1.10596,-0.200297,-0.791497,?,bad,neutral,neutral,1A3S,1A3S,A,?,A,1,1,2.8,Core,A,?,A,1,1,2.8,Surface,4W5V,3UIP,A,?,A,1,1,2.5,Core,?,?,A,?,A,1,0.993671,2.29,Protein interaction,D,?,P63279,P63279,26,1a3s,A,26,A,1,1,2.8,Core,?,?,P63279,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686814,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,L,47,I,L,0,1,0,'Affinity for DNA',bad,neutral,bad,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?,A930848,A930848:1:VHED_BPFD,35.0,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A930848,6.59,3.26,6.59,3.26
686815,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,L,47,I,M,0,1,0,'Affinity for DNA',bad,neutral,bad,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?,A930848,A930848:1:VHED_BPFD,35.0,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A930848,6.59,3.26,6.59,3.26
686816,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,M,47,I,L,0,1,0,'Affinity for DNA',bad,neutral,bad,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?,A930848,A930848:1:VHED_BPFD,35.0,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A930848,6.59,3.26,6.59,3.26
686817,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,M,47,I,M,0,1,0,'Affinity for DNA',bad,neutral,bad,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?,A930848,A930848:1:VHED_BPFD,35.0,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A930848,6.59,3.26,6.59,3.26


In [539]:
df.drop_duplicates(subset=['#Pair_Type', 'Dataset', 'Uniprot_AC', 
                               'Pos_A', 'A1', 'A2', 'Score_A',
       'Pos_C', 'C1', 'C2', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC', 
       ])

,#Pair_Type,Dataset,Uniprot_AC,Pos_A,A1,A2,Pos_C,C1,C2,Score_A,Score_C,Score_AC,PMD_Phen,Effect_A,Effect_C,Effect_AC,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C,Uniprot_AC_2,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID,Rec_Dist_ca,Rec_Dist_closest,Max_Dist_ca,Max_Dist_closest
0,no_type,maveDB_2,P63279,106,A,E,74,K,D,-3.15209,0.421136,-4.32193,?,bad,neutral,bad,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,K,1,1,2.8,Surface,3UIP,3UIP,A,?,A,1,0.993671,2.29,Protein interaction,D,?,A,?,K,1,0.993671,2.29,Protein interaction,C,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
1,no_type,maveDB_2,P63279,106,A,I,67,D,T,-1.46562,-0.649785,-2.00309,?,bad,neutral,bad,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,D,1,1,2.8,Surface,4W5V,4W5V,A,?,A,1,1,2.5,Surface,?,?,A,?,D,1,1,2.5,Surface,?,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
2,no_type,maveDB_2,P63279,106,A,T,78,E,G,-0.200297,-0.0721246,0.71154,?,neutral,neutral,neutral,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,E,1,1,2.8,Surface,4W5V,4W5V,A,?,A,1,1,2.5,Surface,?,?,A,?,E,1,1,2.5,Surface,?,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
3,no_type,maveDB_2,P63279,106,A,T,40,N,D,-0.200297,0.16965,-0.0213069,?,neutral,neutral,neutral,1A3S,1A3S,A,?,A,1,1,2.8,Surface,A,?,N,1,1,2.8,Surface,4W5V,4W5V,A,?,A,1,1,2.5,Surface,?,?,A,?,N,1,1,2.5,Surface,?,?,P63279,P63279,106,1a3s,A,106,A,1,1,2.8,Surface,?,?,P63279,?,?,?,?
4,no_type,maveDB_2,P63279,26,A,G,106,A,T,-1.10596,-0.200297,-0.791497,?,bad,neutral,neutral,1A3S,1A3S,A,?,A,1,1,2.8,Core,A,?,A,1,1,2.8,Surface,4W5V,3UIP,A,?,A,1,1,2.5,Core,?,?,A,?,A,1,0.993671,2.29,Protein interaction,D,?,P63279,P63279,26,1a3s,A,26,A,1,1,2.8,Core,?,?,P63279,?,?,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686814,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,L,47,I,L,0,1,0,'Affinity for DNA',bad,neutral,bad,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?,A930848,A930848:1:VHED_BPFD,35.0,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A930848,6.59,3.26,6.59,3.26
686815,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,L,47,I,M,0,1,0,'Affinity for DNA',bad,neutral,bad,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?,A930848,A930848:1:VHED_BPFD,35.0,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A930848,6.59,3.26,6.59,3.26
686816,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,M,47,I,L,0,1,0,'Affinity for DNA',bad,neutral,bad,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?,A930848,A930848:1:VHED_BPFD,35.0,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A930848,6.59,3.26,6.59,3.26
686817,ExplicitControl,PMD_PMD,A930848:1:VHED_BPFD,35,V,M,47,I,M,0,1,0,'Affinity for DNA',bad,neutral,bad,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,A,47,I,1.0,1.0,2.3,Core,2GN5,2GN5,A,35,V,1.0,1.0,2.3,Core,?,?,A,47,I,1.0,1.0,2.3,Core,?,?,A930848,A930848:1:VHED_BPFD,35.0,2gn5,A,35,V,1.0,1.0,2.3,Core,?,?,A930848,6.59,3.26,6.59,3.26


In [540]:
df.columns

Index(['#Pair_Type', 'Dataset', 'Uniprot_AC', 'Pos_A', 'A1', 'A2', 'Pos_C',
       'C1', 'C2', 'Score_A', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC', 'Max_PDB_ID_A', 'Max_PDB_ID_C', 'Max_Chain_A',
       'Max_Residue-ID_A', 'Max_PDB_AA_A', 'Max_Sequence_identity_A',
       'Max_Coverage_A', 'Max_Resolution_A', 'Max_RIN_A', 'Max_Chain_C',
       'Max_Residue-ID_C', 'Max_PDB_AA_C', 'Max_Sequence_identity_C',
       'Max_Coverage_C', 'Max_Resolution_C', 'Max_RIN_C', 'Rec_PDB_ID_A',
       'Rec_PDB_ID_C', 'Rec_Chain_A', 'Rec_Residue-ID_A', 'Rec_PDB_AA_A',
       'Rec_Sequence_identity_A', 'Rec_Coverage_A', 'Rec_Resolution_A',
       'Rec_RIN_A', 'Rec_Chain_partners_A', 'Rec_Small_molecules_partners_A',
       'Rec_Chain_C', 'Rec_Residue-ID_C', 'Rec_PDB_AA_C',
       'Rec_Sequence_identity_C', 'Rec_Coverage_C', 'Rec_Resolution_C',
       'Rec_RIN_C', 'Rec_Chain_partners_C', 'Rec_Small_molecules_partners_C',
       'Uniprot_AC_2', 'Protein-ID', 'Position', 

In [541]:
df.drop(['Uniprot_AC_2', 'Protein-ID', 'Position', 'PDB-ID', 'Chain',
       'Residue-ID', 'Residue AA type', 'Sequence identity', 'Coverage',
       'Resolution', 'Residue RIN simple classification',
       'Chain type interaction partners',
       'Small molecules type interaction partners', 'Input-ID',], axis=1, inplace=True)

In [544]:
df.columns

Index(['#Pair_Type', 'Dataset', 'Uniprot_AC', 'Pos_A', 'A1', 'A2', 'Pos_C',
       'C1', 'C2', 'Score_A', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC', 'Max_PDB_ID_A', 'Max_PDB_ID_C', 'Max_Chain_A',
       'Max_Residue-ID_A', 'Max_PDB_AA_A', 'Max_Sequence_identity_A',
       'Max_Coverage_A', 'Max_Resolution_A', 'Max_RIN_A', 'Max_Chain_C',
       'Max_Residue-ID_C', 'Max_PDB_AA_C', 'Max_Sequence_identity_C',
       'Max_Coverage_C', 'Max_Resolution_C', 'Max_RIN_C', 'Rec_PDB_ID_A',
       'Rec_PDB_ID_C', 'Rec_Chain_A', 'Rec_Residue-ID_A', 'Rec_PDB_AA_A',
       'Rec_Sequence_identity_A', 'Rec_Coverage_A', 'Rec_Resolution_A',
       'Rec_RIN_A', 'Rec_Chain_partners_A', 'Rec_Small_molecules_partners_A',
       'Rec_Chain_C', 'Rec_Residue-ID_C', 'Rec_PDB_AA_C',
       'Rec_Sequence_identity_C', 'Rec_Coverage_C', 'Rec_Resolution_C',
       'Rec_RIN_C', 'Rec_Chain_partners_C', 'Rec_Small_molecules_partners_C',
       'Rec_Dist_ca', 'Rec_Dist_closest', 'Max_Di

In [545]:
df.drop_duplicates(subset=['#Pair_Type', 'Dataset', 'Uniprot_AC', 
                               'Pos_A', 'A1', 'A2', 'Score_A',
       'Pos_C', 'C1', 'C2', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC', 'Max_PDB_ID_A', 'Max_Chain_A',
       'Max_Residue-ID_A', 'Max_PDB_AA_A', 'Max_Sequence_identity_A',
       'Max_Coverage_A', 'Max_Resolution_A', 'Max_PDB_ID_C', 'Max_Chain_C',
       'Max_Residue-ID_C', 'Max_PDB_AA_C', 'Max_Sequence_identity_C',
       'Max_Coverage_C', 'Max_Resolution_C', 'Max_Dist_ca', 'Max_Dist_closest',
       ]).Dataset.value_counts()

maveDB_15        535917
maveDB_13         51899
maveDB_14         36511
maveDB_4          19360
maveDB_17         17290
maveDB_8           4213
PMD_PMD            2434
maveDB_6           2152
maveDB_2           1151
PMD_Poon            346
CAGI                282
Jacquier_2013       167
Name: Dataset, dtype: int64

In [546]:
df[(df['Max_RIN_A']=='?')&(df['Rec_RIN_A']!='?')]

,#Pair_Type,Dataset,Uniprot_AC,Pos_A,A1,A2,Pos_C,C1,C2,Score_A,Score_C,Score_AC,PMD_Phen,Effect_A,Effect_C,Effect_AC,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C,Rec_Dist_ca,Rec_Dist_closest,Max_Dist_ca,Max_Dist_closest


In [547]:
df[(df['Max_RIN_C']=='?')&(df['Rec_RIN_C']!='?')]

,#Pair_Type,Dataset,Uniprot_AC,Pos_A,A1,A2,Pos_C,C1,C2,Score_A,Score_C,Score_AC,PMD_Phen,Effect_A,Effect_C,Effect_AC,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C,Rec_Dist_ca,Rec_Dist_closest,Max_Dist_ca,Max_Dist_closest


In [548]:
df['Rec_RIN_C'].value_counts()

Protein interaction    376413
Surface                164159
?                       78368
ligand interaction      56049
Core                    11703
DNA interaction            74
metal interaction          32
ion interaction            17
RNA interaction             4
Name: Rec_RIN_C, dtype: int64

In [549]:
for elem in ['Score_A', 'Score_C', 'Score_AC']:
    df[elem] = df[elem].replace([-np.inf], -5)
    df[elem] = df[elem].replace([np.inf], 5)

In [550]:
df = df.fillna('?')
df = df.replace('-', '?')
df['Rec_A1_match'] = (df['A1']==df['Rec_PDB_AA_A']).astype(int)
df['Rec_C1_match'] = (df['C1']==df['Rec_PDB_AA_C']).astype(int)
df['Max_Iden_A1 match'] = (df['A1']==df['Max_PDB_AA_A']).astype(int)
df['Max_Iden_C1_match'] = (df['C1']==df['Max_PDB_AA_C']).astype(int)

In [551]:
for elem in ['Max_PDB_ID_A', 'Max_PDB_ID_C', 'Rec_PDB_ID_A', 'Rec_PDB_ID_C']:
    df[elem] = df[elem].str.split('_',1).str[0]
    df[elem] = df[elem].str.lower()

## short rin

In [552]:
def short_RIN(RIN):
    if RIN in ['Surface', 'Core']:
        return RIN
    elif RIN == 'Protein interaction':
        return 'PPI'
    elif RIN in ['?', 'Disordered']:
        return 'Unknown'
    else:
        return 'MiscInt'

In [553]:
for elem in ['Max_RIN_A', 'Max_RIN_C', 'Rec_RIN_A', 'Rec_RIN_C']:
    #df[elem] = df[elem].str.split('_',1).str[0]
    df['Short_'+elem] = df[elem].apply(short_RIN)

In [554]:
df.columns

Index(['#Pair_Type', 'Dataset', 'Uniprot_AC', 'Pos_A', 'A1', 'A2', 'Pos_C',
       'C1', 'C2', 'Score_A', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC', 'Max_PDB_ID_A', 'Max_PDB_ID_C', 'Max_Chain_A',
       'Max_Residue-ID_A', 'Max_PDB_AA_A', 'Max_Sequence_identity_A',
       'Max_Coverage_A', 'Max_Resolution_A', 'Max_RIN_A', 'Max_Chain_C',
       'Max_Residue-ID_C', 'Max_PDB_AA_C', 'Max_Sequence_identity_C',
       'Max_Coverage_C', 'Max_Resolution_C', 'Max_RIN_C', 'Rec_PDB_ID_A',
       'Rec_PDB_ID_C', 'Rec_Chain_A', 'Rec_Residue-ID_A', 'Rec_PDB_AA_A',
       'Rec_Sequence_identity_A', 'Rec_Coverage_A', 'Rec_Resolution_A',
       'Rec_RIN_A', 'Rec_Chain_partners_A', 'Rec_Small_molecules_partners_A',
       'Rec_Chain_C', 'Rec_Residue-ID_C', 'Rec_PDB_AA_C',
       'Rec_Sequence_identity_C', 'Rec_Coverage_C', 'Rec_Resolution_C',
       'Rec_RIN_C', 'Rec_Chain_partners_C', 'Rec_Small_molecules_partners_C',
       'Rec_Dist_ca', 'Rec_Dist_closest', 'Max_Di

In [555]:
df = df[['#Pair_Type', 'Dataset', 'Uniprot_AC', 'Pos_A', 'A1', 'A2', 
         'Pos_C', 'C1', 'C2', 'Score_A', 'Score_C', 'Score_AC', 'PMD_Phen', 'Effect_A',
       'Effect_C', 'Effect_AC', 
         'Max_PDB_ID_A','Max_Chain_A', 'Max_Residue-ID_A', 'Max_PDB_AA_A', 'Max_Sequence_identity_A',
       'Max_Coverage_A', 'Max_Resolution_A', 'Max_RIN_A',  'Max_Iden_A1 match','Short_Max_RIN_A',
         
         'Max_PDB_ID_C',  'Max_Chain_C','Max_Residue-ID_C', 'Max_PDB_AA_C', 'Max_Sequence_identity_C',
       'Max_Coverage_C', 'Max_Resolution_C', 'Max_RIN_C','Max_Iden_C1_match', 'Short_Max_RIN_C',
         'Max_Dist_ca', 'Max_Dist_closest',
          'Rec_PDB_ID_A', 'Rec_Chain_A', 'Rec_Residue-ID_A', 'Rec_PDB_AA_A',
       'Rec_Sequence_identity_A', 'Rec_Coverage_A', 'Rec_Resolution_A',
       'Rec_RIN_A', 'Rec_Chain_partners_A', 'Rec_Small_molecules_partners_A','Rec_A1_match',
           'Short_Rec_RIN_A',
         'Rec_PDB_ID_C',    'Rec_Chain_C', 'Rec_Residue-ID_C', 'Rec_PDB_AA_C',
       'Rec_Sequence_identity_C', 'Rec_Coverage_C', 'Rec_Resolution_C',
       'Rec_RIN_C', 'Rec_Chain_partners_C', 'Rec_Small_molecules_partners_C',  'Rec_C1_match', 
          'Short_Rec_RIN_C',
       'Rec_Dist_ca', 'Rec_Dist_closest',       
      ]]

In [556]:
for elem in df.columns:
    df = df.rename(columns={elem:elem.replace('_','.')})
    

In [557]:
len(df.drop_duplicates(subset=['#Pair.Type', 'Dataset', 'Uniprot.AC', 
                               'Pos.A', 'A1', 'A2', 'Score.A',
       'Pos.C', 'C1', 'C2', 'Score.C', 'Score.AC', 'PMD.Phen', 'Effect.A',
       'Effect.C', 'Effect.AC', 'Max.PDB.ID.A', 'Max.Chain.A',
       'Max.Residue-ID.A', 'Max.PDB.AA.A', 'Max.Sequence.identity.A',
       'Max.Coverage.A', 'Max.Resolution.A', 'Max.PDB.ID.C', 'Max.Chain.C',
       'Max.Residue-ID.C', 'Max.PDB.AA.C', 'Max.Sequence.identity.C',
       'Max.Coverage.C', 'Max.Resolution.C', 'Max.Dist.ca', 'Max.Dist.closest',
                              ]))

671722

In [558]:
len(df)

686819

In [559]:
df.drop_duplicates(subset=['#Pair.Type', 'Dataset', 'Uniprot.AC', 
                               'Pos.A', 'A1', 'A2', 'Score.A',
       'Pos.C', 'C1', 'C2', 'Score.C', 'Score.AC', 'PMD.Phen', 'Effect.A',
       'Effect.C', 'Effect.AC', 'Max.PDB.ID.A', 'Max.Chain.A',
       'Max.Residue-ID.A', 'Max.PDB.AA.A', 'Max.Sequence.identity.A',
       'Max.Coverage.A', 'Max.Resolution.A', 'Max.PDB.ID.C', 'Max.Chain.C',
       'Max.Residue-ID.C', 'Max.PDB.AA.C', 'Max.Sequence.identity.C',
       'Max.Coverage.C', 'Max.Resolution.C', 'Max.Dist.ca', 'Max.Dist.closest',
                              ])['Dataset'].value_counts()

maveDB_15        535917
maveDB_13         51899
maveDB_14         36511
maveDB_4          19360
maveDB_17         17290
maveDB_8           4213
PMD_PMD            2434
maveDB_6           2152
maveDB_2           1151
PMD_Poon            346
CAGI                282
Jacquier_2013       167
Name: Dataset, dtype: int64

In [561]:
df.to_csv('../../CPD_data_distance_30_03_21_3.tsv.gz',sep='\t',index=False)

In [267]:
# убрать дупликаты в цепях

In [ ]:
# short rin
# правильный порядок колонок
# правильные имена колонок
# AA match

In [327]:
df[(df['Max.RIN.A'].isin(['Surface', 'Core']))&(df['Rec.RIN.A']=='?')]

,#Pair.Type,Dataset,Uniprot.AC,Pos.A,A1,A2,Pos.C,C1,C2,Score.A,Score.C,Score.AC,PMD.Phen,Effect.A,Effect.C,Effect.AC,Max.PDB.ID.A,Max.Chain.A,Max.Residue-ID.A,Max.PDB.AA.A,Max.Sequence.identity.A,Max.Coverage.A,Max.Resolution.A,Max.RIN.A,Max.Iden.A1 match,Short.Max.RIN.A,Max.PDB.ID.C,Max.Chain.C,Max.Residue-ID.C,Max.PDB.AA.C,Max.Sequence.identity.C,Max.Coverage.C,Max.Resolution.C,Max.RIN.C,Max.Iden.C1.match,Short.Max.RIN.C,Max.Dist.ca,Max.Dist.closest,Rec.PDB.ID.A,Rec.Chain.A,Rec.Residue-ID.A,Rec.PDB.AA.A,Rec.Sequence.identity.A,Rec.Coverage.A,Rec.Resolution.A,Rec.RIN.A,Rec.Chain.partners.A,Rec.Small.molecules.partners.A,Rec.A1.match,Short.Rec.RIN.A,Rec.PDB.ID.C,Rec.Chain.C,Rec.Residue-ID.C,Rec.PDB.AA.C,Rec.Sequence.identity.C,Rec.Coverage.C,Rec.Resolution.C,Rec.RIN.C,Rec.Chain.partners.C,Rec.Small.molecules.partners.C,Rec.C1.match,Short.Rec.RIN.C,Rec.Dist.ca,Rec.Dist.closest
10243,no_type,maveDB_6,P38398,221,A,T,175,R,K,-0.360373,0.513977,-1.08636,?,neutral,neutral,bad,7jzv,A,342,T,0.630542,0.106116,3.9,Core,0,Core,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?
10270,no_type,maveDB_6,P38398,224,A,T,164,T,P,-0.242141,1.10486,-1.13099,?,neutral,good,bad,7jzv,A,345,Y,0.630542,0.106116,3.9,Surface,0,Surface,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?
10291,no_type,maveDB_6,P38398,139,Q,I,253,A,V,-1.42468,-0.863129,-0.397386,?,bad,neutral,neutral,7jzv,A,211,S,0.630542,0.106116,3.9,Surface,0,Surface,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?
10295,no_type,maveDB_6,P38398,157,S,C,253,A,V,-0.917743,-0.863129,0.135332,?,neutral,neutral,neutral,7jzv,A,236,T,0.630542,0.106116,3.9,Surface,0,Surface,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?
10301,no_type,maveDB_6,P38398,161,T,E,254,A,V,0.486105,0.632512,-0.0424549,?,neutral,neutral,neutral,7jzv,A,253,T,0.630542,0.106116,3.9,Surface,1,Surface,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686735,Wonder,PMD_PMD,A930527:1:LEP_ECOLI,39,Q,K,31,R,K,0,0,1,'P-2 domain translocation to periplasm',bad,bad,neutral,4wvg,A,355,N,0.39011,0.277863,2.05,Core,0,Core,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?
686782,ExplicitControl,PMD_PMD,A940701:1:LIPL_HUMAN,78,N,Q,397,N,Q,0,1,0,'Secretion',bad,neutral,bad,6oaz,A,70,N,0.47931899999999994,0.815476,3.04,Surface,1,Surface,6oaz,A,386,N,0.47931899999999994,0.815476,3.04,Surface,1,Surface,66.26,63.61,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?
686788,ExplicitControl,PMD_PMD,A890470:1:MOS_MSVMO,358,S,A,263,S,A,0,1,0,'Transforming activity',bad,neutral,bad,6sg4,A,205,G,0.388571,0.37961,2.43,Core,0,Core,6sg4,A,154,V,0.388571,0.37961,2.43,Protein interaction,0,PPI,32.54,31.98,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?
686789,GeneralControl,PMD_PMD,A890470:1:MOS_MSVMO,358,S,A,263,S,A,0,0,0,'In vitro autophosphorylation',bad,bad,bad,6sg4,A,205,G,0.388571,0.37961,2.43,Core,0,Core,6sg4,A,154,V,0.388571,0.37961,2.43,Protein interaction,0,PPI,32.54,31.98,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?,?,?,?,?,?,?,?,?,0,Unknown,?,?


In [328]:
len(df)

686819

# check seq identity

In [329]:
df.columns

Index(['#Pair.Type', 'Dataset', 'Uniprot.AC', 'Pos.A', 'A1', 'A2', 'Pos.C',
       'C1', 'C2', 'Score.A', 'Score.C', 'Score.AC', 'PMD.Phen', 'Effect.A',
       'Effect.C', 'Effect.AC', 'Max.PDB.ID.A', 'Max.Chain.A',
       'Max.Residue-ID.A', 'Max.PDB.AA.A', 'Max.Sequence.identity.A',
       'Max.Coverage.A', 'Max.Resolution.A', 'Max.RIN.A', 'Max.Iden.A1 match',
       'Short.Max.RIN.A', 'Max.PDB.ID.C', 'Max.Chain.C', 'Max.Residue-ID.C',
       'Max.PDB.AA.C', 'Max.Sequence.identity.C', 'Max.Coverage.C',
       'Max.Resolution.C', 'Max.RIN.C', 'Max.Iden.C1.match', 'Short.Max.RIN.C',
       'Max.Dist.ca', 'Max.Dist.closest', 'Rec.PDB.ID.A', 'Rec.Chain.A',
       'Rec.Residue-ID.A', 'Rec.PDB.AA.A', 'Rec.Sequence.identity.A',
       'Rec.Coverage.A', 'Rec.Resolution.A', 'Rec.RIN.A',
       'Rec.Chain.partners.A', 'Rec.Small.molecules.partners.A',
       'Rec.A1.match', 'Short.Rec.RIN.A', 'Rec.PDB.ID.C', 'Rec.Chain.C',
       'Rec.Residue-ID.C', 'Rec.PDB.AA.C', 'Rec.Sequence.identity.C',
 

In [332]:
df1 = df[(df['Max.PDB.ID.A']!='?')&(df['Rec.PDB.ID.A']!='?')&(
    df['Max.PDB.ID.C']!='?')&(df['Rec.PDB.ID.C']!='?')].copy()

In [334]:
for elem in ['Max.Sequence.identity.C','Max.Sequence.identity.A',
             'Rec.Sequence.identity.C','Rec.Sequence.identity.A']:
    df1[elem] = df1[elem].astype(float)

In [338]:
df2 = df1[df1['Rec.Sequence.identity.A']>df1['Max.Sequence.identity.A']].copy()
df2.shape

(7614, 64)

In [339]:
df2[df2['Max.PDB.ID.C']==df2['Max.PDB.ID.A']]

,#Pair.Type,Dataset,Uniprot.AC,Pos.A,A1,A2,Pos.C,C1,C2,Score.A,Score.C,Score.AC,PMD.Phen,Effect.A,Effect.C,Effect.AC,Max.PDB.ID.A,Max.Chain.A,Max.Residue-ID.A,Max.PDB.AA.A,Max.Sequence.identity.A,Max.Coverage.A,Max.Resolution.A,Max.RIN.A,Max.Iden.A1 match,Short.Max.RIN.A,Max.PDB.ID.C,Max.Chain.C,Max.Residue-ID.C,Max.PDB.AA.C,Max.Sequence.identity.C,Max.Coverage.C,Max.Resolution.C,Max.RIN.C,Max.Iden.C1.match,Short.Max.RIN.C,Max.Dist.ca,Max.Dist.closest,Rec.PDB.ID.A,Rec.Chain.A,Rec.Residue-ID.A,Rec.PDB.AA.A,Rec.Sequence.identity.A,Rec.Coverage.A,Rec.Resolution.A,Rec.RIN.A,Rec.Chain.partners.A,Rec.Small.molecules.partners.A,Rec.A1.match,Short.Rec.RIN.A,Rec.PDB.ID.C,Rec.Chain.C,Rec.Residue-ID.C,Rec.PDB.AA.C,Rec.Sequence.identity.C,Rec.Coverage.C,Rec.Resolution.C,Rec.RIN.C,Rec.Chain.partners.C,Rec.Small.molecules.partners.C,Rec.C1.match,Short.Rec.RIN.C,Rec.Dist.ca,Rec.Dist.closest
16837,no_type,maveDB_13,Q9ES00,1099,A,D,1096,Y,C,-0.266066,-0.0190026,-1.41683,?,neutral,neutral,bad,2kr4,A,11,A,0.964706,0.07246380000000001,2.5,Core,1,Core,2kr4,A,8,Y,0.964706,0.07246380000000001,2.5,Surface,1,Surface,5.63,3.03,5o75,A,1099,A,0.971831,0.060528599999999995,1.48,Core,?,?,1,Core,2kre,A,1225,Y,0.940000,0.0852515,2.5,Core,?,?,1,Core,?,?
16847,no_type,maveDB_13,Q9ES00,1099,A,D,1096,Y,F,-0.266066,0.023491,-0.00132387,?,neutral,neutral,neutral,2kr4,A,11,A,0.964706,0.07246380000000001,2.5,Core,1,Core,2kr4,A,8,Y,0.964706,0.07246380000000001,2.5,Surface,1,Surface,5.63,3.03,5o75,A,1099,A,0.971831,0.060528599999999995,1.48,Core,?,?,1,Core,2kre,A,1225,Y,0.940000,0.0852515,2.5,Core,?,?,1,Core,?,?
16857,no_type,maveDB_13,Q9ES00,1099,A,D,1096,Y,S,-0.266066,0.362869,-0.55742,?,neutral,neutral,neutral,2kr4,A,11,A,0.964706,0.07246380000000001,2.5,Core,1,Core,2kr4,A,8,Y,0.964706,0.07246380000000001,2.5,Surface,1,Surface,5.63,3.03,5o75,A,1099,A,0.971831,0.060528599999999995,1.48,Core,?,?,1,Core,2kre,A,1225,Y,0.940000,0.0852515,2.5,Core,?,?,1,Core,?,?
16897,no_type,maveDB_13,Q9ES00,1099,A,G,1096,Y,S,0.156823,0.362869,0.443608,?,neutral,neutral,neutral,2kr4,A,11,A,0.964706,0.07246380000000001,2.5,Core,1,Core,2kr4,A,8,Y,0.964706,0.07246380000000001,2.5,Surface,1,Surface,5.63,3.03,5o75,A,1099,A,0.971831,0.060528599999999995,1.48,Core,?,?,1,Core,2kre,A,1225,Y,0.940000,0.0852515,2.5,Core,?,?,1,Core,?,?
16952,no_type,maveDB_13,Q9ES00,1099,A,P,1096,Y,F,-0.202079,0.023491,-2.20154,?,neutral,neutral,bad,2kr4,A,11,A,0.964706,0.07246380000000001,2.5,Core,1,Core,2kr4,A,8,Y,0.964706,0.07246380000000001,2.5,Surface,1,Surface,5.63,3.03,5o75,A,1099,A,0.971831,0.060528599999999995,1.48,Core,?,?,1,Core,2kre,A,1225,Y,0.940000,0.0852515,2.5,Core,?,?,1,Core,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686694,Compensator,PMD_PMD,A911218:1:TPIS_CHICK,165,E,D,233,G,R,0,1,1,'kcat/Km',bad,neutral,neutral,1spq,A,165,E,0.995885,0.9798389999999999,2.16,Core,1,Core,1spq,A,233,G,0.995885,0.9798389999999999,2.16,Surface,1,Surface,14.03,10.32,1sw3,A,165,E,0.995951,0.9959680000000001,2.03,ligand interaction,?,A:350,1,MiscInt,1sw3,A,233,G,0.995951,0.9959680000000001,2.03,ligand interaction,?,A:350,1,MiscInt,13.75,8.71
686695,Compensator,PMD_PMD,A911218:1:TPIS_CHICK,165,E,D,97,E,D,0,1,1,'kcat/Km',bad,neutral,neutral,1spq,A,165,E,0.995885,0.9798389999999999,2.16,Core,1,Core,1spq,A,97,E,0.995885,0.9798389999999999,2.16,Protein interaction,1,PPI,10.81,5.43,1sw3,A,165,E,0.995951,0.9959680000000001,2.03,ligand interaction,?,A:350,1,MiscInt,1spq,A,97,E,0.995885,0.9798389999999999,2.16,Protein interaction,B,?,1,PPI,?,?
686697,Compensator,PMD_PMD,A911218:1:TPIS_CHICK,165,E,D,96,S,T,0,1,1,'kcat/Km',bad,neutral,neutral,1spq,A,165,E,0.995885,0.9798389999999999,2.16,Core,1,Core,1spq,A,96,S,0.995885,0.9798389999999999,2.16,Core,1,Core,7.58,2.94,1spq,B,165,E,1.000000,0.9637100000000001,2.16,Cor

In [343]:
df2[df2['Rec.PDB.ID.C']==df2['Rec.PDB.ID.A']]

,#Pair.Type,Dataset,Uniprot.AC,Pos.A,A1,A2,Pos.C,C1,C2,Score.A,Score.C,Score.AC,PMD.Phen,Effect.A,Effect.C,Effect.AC,Max.PDB.ID.A,Max.Chain.A,Max.Residue-ID.A,Max.PDB.AA.A,Max.Sequence.identity.A,Max.Coverage.A,Max.Resolution.A,Max.RIN.A,Max.Iden.A1 match,Short.Max.RIN.A,Max.PDB.ID.C,Max.Chain.C,Max.Residue-ID.C,Max.PDB.AA.C,Max.Sequence.identity.C,Max.Coverage.C,Max.Resolution.C,Max.RIN.C,Max.Iden.C1.match,Short.Max.RIN.C,Max.Dist.ca,Max.Dist.closest,Rec.PDB.ID.A,Rec.Chain.A,Rec.Residue-ID.A,Rec.PDB.AA.A,Rec.Sequence.identity.A,Rec.Coverage.A,Rec.Resolution.A,Rec.RIN.A,Rec.Chain.partners.A,Rec.Small.molecules.partners.A,Rec.A1.match,Short.Rec.RIN.A,Rec.PDB.ID.C,Rec.Chain.C,Rec.Residue-ID.C,Rec.PDB.AA.C,Rec.Sequence.identity.C,Rec.Coverage.C,Rec.Resolution.C,Rec.RIN.C,Rec.Chain.partners.C,Rec.Small.molecules.partners.C,Rec.C1.match,Short.Rec.RIN.C,Rec.Dist.ca,Rec.Dist.closest
684120,Compensator,PMD_Poon,POL_HV1A2,823,L,A,699,D,G,0,?,1,?,bad,?,neutral,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,1,MiscInt,1rt4,A,110,D,0.964618,0.373695,2.9,Surface,1,Surface,17.28,15.13,1rtj,B,234,L,0.971831,0.296451,2.35,Core,?,?,1,Core,1rtj,A,110,D,0.965009,0.377871,2.35,Surface,?,?,1,Surface,14.85,11.11
684121,Compensator,PMD_Poon,POL_HV1A2,823,L,A,699,D,G,0,?,1,?,bad,?,neutral,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,1,MiscInt,1rt4,A,110,D,0.964618,0.373695,2.9,Surface,1,Surface,17.28,15.13,1rtj,B,234,L,0.971831,0.296451,2.35,Core,?,?,1,Core,1rtj,B,110,D,0.971831,0.296451,2.35,Core,?,?,1,Core,14.85,11.11
684122,Compensator,PMD_Poon,POL_HV1A2,823,L,A,775,D,V,0,?,1,?,bad,?,neutral,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,1,MiscInt,1rt4,A,186,D,0.964618,0.373695,2.9,Core,1,Core,15.15,11.29,1rtj,A,234,L,0.965009,0.377871,2.35,Core,?,?,1,Core,1rtj,A,186,D,0.965009,0.377871,2.35,Surface,?,?,1,Surface,13.56,9.19
684123,Compensator,PMD_Poon,POL_HV1A2,823,L,A,775,D,V,0,?,1,?,bad,?,neutral,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,1,MiscInt,1rt4,A,186,D,0.964618,0.373695,2.9,Core,1,Core,15.15,11.29,1rtj,A,234,L,0.965009,0.377871,2.35,Core,?,?,1,Core,1rtj,B,186,D,0.971831,0.296451,2.35,Core,?,?,1,Core,13.56,9.19
684126,Compensator,PMD_Poon,POL_HV1A2,823,L,A,991,W,R,0,?,1,?,bad,?,neutral,1tkx,A,234,L,0.964486,0.372303,2.85,ligand interaction,1,MiscInt,1tkx,A,402,W,0.964486,0.372303,2.85,Protein interaction,1,PPI,41.91,38.02,4ifv,B,234,L,0.966019,0.286708,2.05,Core,?,?,1,Core,4ifv,A,402,W,0.955036,0.386917,2.05,Protein interaction,B,?,1,PPI,23.09,18.34
684127,Compensator,PMD_Poon,POL_HV1A2,823,L,A,991,W,R,0,?,1,?,bad,?,neutral,1tkx,A,234,L,0.964486,0.372303,2.85,ligand interaction,1,MiscInt,1tkx,A,402,W,0.964486,0.372303,2.85,Protein interaction,1,PPI,41.91,38.02,4ifv,B,234,L,0.966019,0.286708,2.05,Core,?,?,1,Core,4ifv,B,402,W,0.966019,0.286708,2.05,ligand interaction,?,B:501,1,MiscInt,23.09,18.34
684128,Compensator,PMD_Poon,POL_HV1A2,823,L,A,995,W,R,0,?,1,?,bad,?,neutral,1tkx,A,234,L,0.964486,0.372303,2.85,ligand interaction,1,MiscInt,1tkx,A,406,W,0.964486,0.372303,2.85,Protein interaction,1,PPI,46.34,40.95,1rtj,B,234,L,0.971831,0.296451,2.35,Core,?,?,1,Core,1rtj,A,406,W,0.965009,0.377871,2.35,Protein interaction,B,?,1,PPI,18.18,14.33
684129,Compensator,PMD_Poon,POL_HV1A2,823,L,A,995,W,R,0,?,1,?,bad,?,neutral,1tkx,A,234,L,0.964486,0.372303,2.85,ligand interaction,1,MiscInt,1tkx,A,406,W,0.964486,0.372303,2.85,Protein interaction,1,PPI,46.34,40.95,1rtj,B,234,L,0.971831,0.296451,2.35,Core,?,?,1,Core,1rtj,B,406,W,0.971831,0.296451,2.35,Core,?,?,1,Core,18.18,14.33
684231,no_type,PMD_PMD,A931537:1:A32872,144,A,F,120,F,A,0,0,0,'Transcriptional activation',bad,bad,bad,1mdy,A,153,A,0.897059,0.228188,2.8,Core,1,Core,1mdy,A,129,F,0.897059,0.228188,2.8,Protein interaction,1,PPI,10.59,8.29,1mdy,B,153,A,0.935484,0.208054,2.8,Core,?,?,1,Core,1mdy,A,129,F,0.897059,0.228188,2.8,Protein interaction,B,?,1,PPI,10.42,7.97
684234,no_type,PMD_PMD,A931537:1:A32872,146,R,E,152,Q,E,0,1,0,'Transcriptional activation',bad,neutral,bad,

In [346]:
struct.columns

Index(['Protein-ID', 'Position', 'PDB-ID', 'Chain', 'Residue-ID',
       'Residue AA type', 'Sequence identity', 'Coverage', 'Resolution',
       'Residue RIN simple classification', 'Chain type interaction partners',
       'Small molecules type interaction partners', 'Input-ID'],
      dtype='object')

In [364]:
struct[(struct['Input-ID'].str.contains('POL_HV1A2'))&(struct['Position']==823)
       &(struct['PDB-ID']=='1RT4')].sort_values(
        by='Sequence identity', ascending=False)

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
115533,P03369,823,1RT4,B,234,L,0.972772,0.281141,2.9,Core,NaN,NaN,POL_HV1A2
113762,P03369,823,1RT4,A,234,L,0.964618,0.373695,2.9,ligand interaction,NaN,A:999,POL_HV1A2


In [357]:
struct[(struct['Input-ID'].str.contains('POL_HV1A2'))&(struct['Position']==699)].sort_values(
        by='Sequence identity', ascending=False)

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
115592,P03369,699,1TKX,B,110,D,0.972772,0.281141,2.85,Surface,NaN,NaN,POL_HV1A2
115537,P03369,699,1RT4,B,110,D,0.972772,0.281141,2.90,Surface,NaN,NaN,POL_HV1A2
115726,P03369,699,2OPP,B,110,D,0.972705,0.280445,2.55,Core,NaN,NaN,POL_HV1A2
115625,P03369,699,3DLE,B,110,D,0.972705,0.280445,2.50,Surface,NaN,NaN,POL_HV1A2
115655,P03369,699,1RT2,B,110,D,0.972637,0.279749,2.55,Surface,NaN,NaN,POL_HV1A2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116301,P03369,699,1MU2,B,110,D,0.609453,0.279749,2.35,Core,NaN,NaN,POL_HV1A2
116308,P03369,699,5OVN,A,109,D,0.497132,0.362188,2.94,Surface,NaN,NaN,POL_HV1A2
116315,P03369,699,5OVN,B,109,D,0.485149,0.280946,2.94,Surface,NaN,NaN,POL_HV1A2
116319,P03369,699,1D0E,A,150,D,0.356275,0.170462,3.00,Core,NaN,NaN,POL_HV1A2


In [362]:
posit = pd.read_csv('../protein_positions_with_max_rec2_pdb.tsv',sep='\t')
posit[(posit['Uniprot_AC'].str.contains('POL_HV1A2'))&(posit['Pos_A']==823)]

,Uniprot_AC,Pos_A,Pos_C,Max_PDB_ID_A,Max_PDB_ID_C,Max_Chain_A,Max_Residue-ID_A,Max_PDB_AA_A,Max_Sequence_identity_A,Max_Coverage_A,Max_Resolution_A,Max_RIN_A,Max_Chain_C,Max_Residue-ID_C,Max_PDB_AA_C,Max_Sequence_identity_C,Max_Coverage_C,Max_Resolution_C,Max_RIN_C,Rec_PDB_ID_A,Rec_PDB_ID_C,Rec_Chain_A,Rec_Residue-ID_A,Rec_PDB_AA_A,Rec_Sequence_identity_A,Rec_Coverage_A,Rec_Resolution_A,Rec_RIN_A,Rec_Chain_partners_A,Rec_Small_molecules_partners_A,Rec_Chain_C,Rec_Residue-ID_C,Rec_PDB_AA_C,Rec_Sequence_identity_C,Rec_Coverage_C,Rec_Resolution_C,Rec_RIN_C,Rec_Chain_partners_C,Rec_Small_molecules_partners_C
23645,POL_HV1A2,823,699,1rt4,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,A,110,D,0.964618,0.373695,2.9,Surface,1rtj,1rtj,A,234,L,0.965009,0.377871,2.35,Core,?,?,A,110,D,0.965009,0.377871,2.35,Surface,?,?
23646,POL_HV1A2,823,699,1rt4,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,A,110,D,0.964618,0.373695,2.9,Surface,1rtj,1rtj,A,234,L,0.965009,0.377871,2.35,Core,?,?,B,110,D,0.971831,0.296451,2.35,Core,?,?
23647,POL_HV1A2,823,699,1rt4,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,A,110,D,0.964618,0.373695,2.9,Surface,1rtj,1rtj,B,234,L,0.971831,0.296451,2.35,Core,?,?,A,110,D,0.965009,0.377871,2.35,Surface,?,?
23648,POL_HV1A2,823,699,1rt4,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,A,110,D,0.964618,0.373695,2.9,Surface,1rtj,1rtj,B,234,L,0.971831,0.296451,2.35,Core,?,?,B,110,D,0.971831,0.296451,2.35,Core,?,?
23649,POL_HV1A2,823,699,1rt4,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,B,110,D,0.972772,0.281141,2.9,Surface,1rtj,1rtj,A,234,L,0.965009,0.377871,2.35,Core,?,?,A,110,D,0.965009,0.377871,2.35,Surface,?,?
23650,POL_HV1A2,823,699,1rt4,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,B,110,D,0.972772,0.281141,2.9,Surface,1rtj,1rtj,A,234,L,0.965009,0.377871,2.35,Core,?,?,B,110,D,0.971831,0.296451,2.35,Core,?,?
23651,POL_HV1A2,823,699,1rt4,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,B,110,D,0.972772,0.281141,2.9,Surface,1rtj,1rtj,B,234,L,0.971831,0.296451,2.35,Core,?,?,A,110,D,0.965009,0.377871,2.35,Surface,?,?
23652,POL_HV1A2,823,699,1rt4,1rt4,A,234,L,0.964618,0.373695,2.9,ligand interaction,B,110,D,0.972772,0.281141,2.9,Surface,1rtj,1rtj,B,234,L,0.971831,0.296451,2.35,Core,?,?,B,110,D,0.971831,0.296451,2.35,Core,?,?
23653,POL_HV1A2,823,699,1rt4,1rt4,B,234,L,0.972772,0.281141,2.9,Core,A,110,D,0.964618,0.373695,2.9,Surface,1rtj,1rtj,A,234,L,0.965009,0.377871,2.35,Core,?,?,A,110,D,0.965009,0.377871,2.35,Surface,?,?
23654,POL_HV1A2,823,699,1rt4,1rt4,B,234,L,0.972772,0.281141,2.9,Core,A,110,D,0.964618,0.373695,2.9,Surface,1rtj,1rtj,A,234,L,0.965009,0.377871,2.35,Core,?,?,B,110,D,0.971831,0.296451,2.35,Core,?,?


## проверка RIN

In [367]:
df.columns

Index(['#Pair.Type', 'Dataset', 'Uniprot.AC', 'Pos.A', 'A1', 'A2', 'Pos.C',
       'C1', 'C2', 'Score.A', 'Score.C', 'Score.AC', 'PMD.Phen', 'Effect.A',
       'Effect.C', 'Effect.AC', 'Max.PDB.ID.A', 'Max.Chain.A',
       'Max.Residue-ID.A', 'Max.PDB.AA.A', 'Max.Sequence.identity.A',
       'Max.Coverage.A', 'Max.Resolution.A', 'Max.RIN.A', 'Max.Iden.A1 match',
       'Short.Max.RIN.A', 'Max.PDB.ID.C', 'Max.Chain.C', 'Max.Residue-ID.C',
       'Max.PDB.AA.C', 'Max.Sequence.identity.C', 'Max.Coverage.C',
       'Max.Resolution.C', 'Max.RIN.C', 'Max.Iden.C1.match', 'Short.Max.RIN.C',
       'Max.Dist.ca', 'Max.Dist.closest', 'Rec.PDB.ID.A', 'Rec.Chain.A',
       'Rec.Residue-ID.A', 'Rec.PDB.AA.A', 'Rec.Sequence.identity.A',
       'Rec.Coverage.A', 'Rec.Resolution.A', 'Rec.RIN.A',
       'Rec.Chain.partners.A', 'Rec.Small.molecules.partners.A',
       'Rec.A1.match', 'Short.Rec.RIN.A', 'Rec.PDB.ID.C', 'Rec.Chain.C',
       'Rec.Residue-ID.C', 'Rec.PDB.AA.C', 'Rec.Sequence.identity.C',
 

In [368]:
df1 = df[(df['Short.Max.RIN.A']=='MiscInt')&(df['Short.Rec.RIN.A']=='Core')].copy()

In [375]:
df1[['Max.PDB.ID.A','Max.PDB.ID.C','Rec.PDB.ID.A','Rec.PDB.ID.C',
     'Short.Max.RIN.A','Short.Max.RIN.C','Short.Rec.RIN.A','Short.Rec.RIN.C',
    'Pos.A','Pos.C']]

,Max.PDB.ID.A,Max.PDB.ID.C,Rec.PDB.ID.A,Rec.PDB.ID.C,Short.Max.RIN.A,Short.Max.RIN.C,Short.Rec.RIN.A,Short.Rec.RIN.C,Pos.A,Pos.C
683599,1ero,1ero,4gku,4gku,MiscInt,Surface,Core,Core,68,180
683600,1ero,1ero,4gku,4gku,MiscInt,Surface,Core,Core,71,180
683620,1ero,1ero,4gku,4gku,MiscInt,Surface,Core,Core,128,180
683621,1ero,1ero,4gku,4gku,MiscInt,Surface,Core,Core,128,180
683622,1ero,1ero,4gku,4gku,MiscInt,Surface,Core,Core,130,180
683668,1ero,1ero,4gku,4gku,MiscInt,Surface,Core,Core,233,180
683669,1ero,1ero,4gku,4gku,MiscInt,Surface,Core,Core,234,180
683758,1ero,1ero,1s0w,1s0w,MiscInt,Surface,Core,Core,235,180
684120,1rt4,1rt4,1rtj,1rtj,MiscInt,Surface,Core,Surface,823,699
684121,1rt4,1rt4,1rtj,1rtj,MiscInt,Surface,Core,Core,823,699


In [376]:
struct[(struct['Position']==823)
       &(struct['PDB-ID']=='1TKX')].sort_values(
        by='Sequence identity', ascending=False)

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
115588,P03369,823,1TKX,B,234,L,0.972772,0.281141,2.85,Surface,NaN,NaN,POL_HV1A2
113783,P03369,823,1TKX,A,234,L,0.964486,0.372303,2.85,ligand interaction,NaN,A:999,POL_HV1A2


# првоерка из RIN graphs

In [387]:
struct[(struct['Residue-ID']==126)
       &(struct['PDB-ID']=='2DJ3')].sort_values(
        by='Sequence identity', ascending=False)

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID


In [391]:
struct[
       (struct['PDB-ID'].str.contains('2DJ3'))].sort_values(
        by='Sequence identity', ascending=False)

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
327405,A910249:2:PDIA4_MOUSE,637,2DJ3_AU,A,126,E,0.952756,0.197205,2.5,Surface,NaN,NaN,A910249
327406,A910249:2:PDIA4_MOUSE,635,2DJ3_AU,A,124,K,0.952756,0.197205,2.5,Surface,NaN,NaN,A910249
499173,A001349:1:[DIRECT],385,2DJ3_AU,A,40,C,0.552000,0.255624,2.5,Core,NaN,NaN,A001349
246446,A920545:1:PDIA1_HUMAN,397,2DJ3_AU,A,37,C,0.478632,0.223282,2.5,Core,NaN,NaN,A920545
276706,A920554:1:PDIA1_HUMAN,397,2DJ3_AU,A,37,C,0.478632,0.223282,2.5,Core,NaN,NaN,A920554


In [398]:
struct[(struct['Residue-ID']=='32')&
       (struct['PDB-ID'].str.contains('2KWD'))].sort_values(
        by='Residue-ID', ascending=False)

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
81124,P06654,258,2KWD,A,32,Q,0.964286,0.125,2.5,Protein interaction,D,NaN,P06654


In [402]:
struct[(struct['Position']==266)&
       (struct['PDB-ID'].str.contains('2KWD'))].sort_values(
        by='Residue-ID', ascending=False)

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
81133,P06654,266,2KWD,A,40,D,0.964286,0.125,2.5,Protein interaction,D,NaN,P06654


In [479]:
len(df)

686819

# find duplicates

In [5]:
df = pd.read_csv('../protein_positions_with_pdb.tsv',sep='\t')
df.head()

,Uniprot_AC,Pos_A,Pos_C,PDB_ID_A,PDB_ID_C,Protein-ID_x,Chain_A,Residue-ID_A,PDB_AA_A,Sequence_identity_A,Coverage_A,Resolution_A,RIN_A,Chain_partners_A,Small_molecules_partners_A,Protein-ID_y,Chain_C,Residue-ID_C,PDB_AA_C,Sequence_identity_C,Coverage_C,Resolution_C,RIN_C,Chain_partners_C,Small_molecules_partners_C
0,P63279,106,74,1A3S,1A3S,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?,P63279,A,74,K,1.0,1.0,2.8,Surface,?,?
1,P63279,106,67,1A3S,1A3S,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?,P63279,A,67,D,1.0,1.0,2.8,Surface,?,?
2,P63279,106,78,1A3S,1A3S,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?,P63279,A,78,E,1.0,1.0,2.8,Surface,?,?
3,P63279,106,40,1A3S,1A3S,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?,P63279,A,40,N,1.0,1.0,2.8,Surface,?,?
4,P63279,26,106,1A3S,1A3S,P63279,A,26,A,1.0,1.0,2.8,Core,?,?,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?


In [6]:
df.shape

(23562, 25)

In [25]:
duplic = df[df.duplicated(subset=['Uniprot_AC','Pos_A', 'Pos_C'])]
duplic.to_csv('duplicated.tsv',sep='\t', index=False)

In [38]:
duplic = pd.read_csv('duplicated.tsv',sep='\t')
duplic = duplic[duplic['PDB_ID']!='?']
duplic

,Uniprot_AC,Pos,PDB_ID,Chain,Residue-ID,PDB_AA,Sequence_identity,Coverage,RIN,Chain_partners,Small_molecules_partners
14,A001711,114,13GS,A,113,A,1.0,1.0,Surface,?,?
15,A001711,114,13GS,B,113,A,1.0,0.990476,Surface,?,?
16,A001719,154,13GS,A,153,Y,1.0,1.0,Core,?,?
17,A001719,154,13GS,B,153,Y,1.0,0.990476,Core,?,?
18,A012263,102,13GS,A,101,C,1.0,1.0,Surface,?,?
...,...,...,...,...,...,...,...,...,...,...,...
1051,A982396,247,7MDH,A,207,C,0.994302,0.818182,Protein interaction,B,?
1052,A940874,405,7MDH,A,365,C,0.994302,0.818182,Core,?,?
1053,A940874,405,7MDH,B,365,C,0.9944440000000001,0.839161,Core,?,?
1054,A982396,69,7MDH,A,29,C,0.994302,0.818182,Surface,?,?


In [41]:
duplic.shape

(1042, 11)

In [64]:
unique = duplic[duplic.duplicated(subset=['Uniprot_AC', 'Pos', 'PDB_ID',])].copy()

unique = unique[['Uniprot_AC', 'Pos', 'PDB_ID','Chain']]

In [65]:
def find_RINS(Uniprot_AC, Pos, PDB_ID):
    local = duplic[(duplic['Uniprot_AC'] == Uniprot_AC)&(
                duplic['Pos'] == Pos)&(
                duplic['PDB_ID'] == PDB_ID)]
    
    rins = local['RIN'].values.tolist()
    return rins

In [66]:
unique['RINS'] = unique.apply(
        lambda x: find_RINS(x.Uniprot_AC, x.Pos,x.PDB_ID),axis=1)

In [67]:
def calculate_rins(rins):
    n = len(set(rins))
    return n


In [68]:
unique['RIN_n'] = unique['RINS'].apply(calculate_rins)

In [69]:
unique['RIN_n'].value_counts()

1    457
2     78
Name: RIN_n, dtype: int64

In [70]:
unique['RIN_total'] = unique['RINS'].apply(len)

In [71]:
unique['RIN_total'].value_counts()

2    397
3     84
4     54
Name: RIN_total, dtype: int64

In [72]:
unique[unique['RIN_n']==2]

,Uniprot_AC,Pos,PDB_ID,Chain,RINS,RIN_n,RIN_total
43,A002694,147,1A4I,A,"[Surface, Core]",2,2
47,A993832,414,1B9B,A,"[Protein interaction, Surface]",2,2
63,A951084,31,1BDT,C,"[Surface, Surface, Protein interaction]",2,3
64,A951084,31,1BDT,B,"[Surface, Surface, Protein interaction]",2,3
126,A970276,63,1F93,B,"[Protein interaction, Surface]",2,2
136,A931445,167,1FG9,D,"[Core, Core, Surface]",2,3
137,A931445,167,1FG9,E,"[Core, Core, Surface]",2,3
142,A960518,818,1FK9,A,"[Core, Surface]",2,2
181,A890534,207,1HCQ,B,"[DNA interaction, Surface]",2,2
215,A921119,66,1JWH,A,"[Core, ligand interaction]",2,2


In [52]:
struct.head()

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
0,A992881:1:[DIRECT],365,4X0F,A,586,F,0.972010,0.837953,3.22,Protein interaction,B,A:804,A992881
1,A992881:1:[DIRECT],354,4X0F,A,575,Y,0.972010,0.837953,3.22,ligand interaction,NaN,A:804,A992881
2,A992881:1:[DIRECT],401,4X0F,A,622,I,0.972010,0.837953,3.22,Protein interaction,B,NaN,A992881
3,A992881:1:[DIRECT],383,4X0F,A,604,A,0.972010,0.837953,3.22,Core,NaN,NaN,A992881
4,A992881:1:[DIRECT],365,4WZI,B,586,F,0.971867,0.807851,2.58,Protein interaction,A,NaN,A992881


In [54]:
unique[unique['RIN_n']==2].shape

(78, 5)

In [57]:
report = pd.merge(unique[unique['RIN_n']==2],struct, how='left',
        left_on=['Uniprot_AC', 'Pos','PDB_ID'],
        right_on = ['Input-ID', 'Position', 'PDB-ID'])

report = report.drop(['Uniprot_AC','Pos','PDB_ID','RINS','RIN_n'],axis=1)

In [59]:
report.to_csv('duplicated_mapping_rins_inconsistence.tsv', sep='\t',index=False)

In [78]:
report.groupby(['Position', 'PDB-ID']).count()['Protein-ID'].value_counts()

2    49
6     8
4     6
3     1
Name: Protein-ID, dtype: int64

In [85]:
report.groupby(['Position', 'PDB-ID', 'Chain']).size()

Position  PDB-ID  Chain
16        6UCY    A        1
                  B        1
31        1BDT    A        2
                  B        2
                  C        2
34        6PXW    C        2
                  E        2
48        6PXW    C        1
                  E        1
49        6PXW    C        2
                  E        2
57        6UCY    A        1
                  B        1
59        1LMB    3        1
                  4        1
63        1F93    A        1
                  B        1
66        1JWH    A        1
                  B        1
77        1QVC    A        2
                  B        2
                  D        2
90        1SCE    A        2
                  C        2
                  D        2
103       1RT4    A        1
                  B        1
114       1THT    A        1
                  B        1
128       5G1E    A        1
                  B        1
130       1MDY    A        1
                  B        1
147       1A4I    A

In [76]:
report

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
0,A002694:1:[DIRECT],147,1A4I,B,147,C,1.000000,0.980066,1.50,Surface,NaN,NaN,A002694
1,A002694:1:[DIRECT],147,1A4I,A,147,C,1.000000,0.946844,1.50,Core,NaN,NaN,A002694
2,A993832:2:PGKT_THEMA,414,1B9B,B,515,K,1.000000,0.389908,2.85,Protein interaction,A,NaN,A993832
3,A993832:2:PGKT_THEMA,414,1B9B,A,15,K,1.000000,0.385321,2.85,Surface,NaN,NaN,A993832
4,A951084:1:RARC_BPP22,31,1BDT,B,31,R,1.000000,1.000000,2.50,Protein interaction,D,NaN,A951084
5,A951084:1:RARC_BPP22,31,1BDT,A,31,R,1.000000,0.981132,2.50,Surface,NaN,NaN,A951084
6,A951084:1:RARC_BPP22,31,1BDT,C,31,R,1.000000,0.943396,2.50,Surface,NaN,NaN,A951084
7,A951084:1:RARC_BPP22,31,1BDT,B,31,R,1.000000,1.000000,2.50,Protein interaction,D,NaN,A951084
8,A951084:1:RARC_BPP22,31,1BDT,A,31,R,1.000000,0.981132,2.50,Surface,NaN,NaN,A951084
9,A951084:1:RARC_BPP22,31,1BDT,C,31,R,1.000000,0.943396,2.50,Surface,NaN,NaN,A951084


In [7]:
df.drop_duplicates(subset=['Uniprot_AC','Pos_A', 'Pos_C'])

,Uniprot_AC,Pos_A,Pos_C,PDB_ID_A,PDB_ID_C,Protein-ID_x,Chain_A,Residue-ID_A,PDB_AA_A,Sequence_identity_A,Coverage_A,Resolution_A,RIN_A,Chain_partners_A,Small_molecules_partners_A,Protein-ID_y,Chain_C,Residue-ID_C,PDB_AA_C,Sequence_identity_C,Coverage_C,Resolution_C,RIN_C,Chain_partners_C,Small_molecules_partners_C
0,P63279,106,74,1A3S,1A3S,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?,P63279,A,74,K,1.0,1.0,2.8,Surface,?,?
1,P63279,106,67,1A3S,1A3S,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?,P63279,A,67,D,1.0,1.0,2.8,Surface,?,?
2,P63279,106,78,1A3S,1A3S,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?,P63279,A,78,E,1.0,1.0,2.8,Surface,?,?
3,P63279,106,40,1A3S,1A3S,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?,P63279,A,40,N,1.0,1.0,2.8,Surface,?,?
4,P63279,26,106,1A3S,1A3S,P63279,A,26,A,1.0,1.0,2.8,Core,?,?,P63279,A,106,A,1.0,1.0,2.8,Surface,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23554,A890140,30,36,1COP_AU,1COP_AU,A890140:1:RCRO_LAMBD,D,30,I,1.0,1.0,2.5,Core,?,?,A890140:1:RCRO_LAMBD,D,36,A,1.0,1.0,2.5,Surface,?,?
23555,A941188,159,160,7JY6,7JY6,A941188:1:RECA_SHIFL,A,158,E,1.0,0.943343,2.5,Surface,?,?,A941188:1:RECA_SHIFL,A,159,I,1.0,0.943343,2.5,Surface,?,?
23559,A015899,42,18,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
23560,A930848,82,45,2GN5,2GN5,A930848:1:VHED_BPFD,A,82,R,1.0,1.0,2.3,Surface,?,?,A930848:1:VHED_BPFD,A,45,V,1.0,1.0,2.3,Surface,?,?


In [16]:
struct

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
0,A992881:1:[DIRECT],365,4X0F,A,586,F,0.972010,0.837953,3.22,Protein interaction,B,A:804,A992881
1,A992881:1:[DIRECT],354,4X0F,A,575,Y,0.972010,0.837953,3.22,ligand interaction,NaN,A:804,A992881
2,A992881:1:[DIRECT],401,4X0F,A,622,I,0.972010,0.837953,3.22,Protein interaction,B,NaN,A992881
3,A992881:1:[DIRECT],383,4X0F,A,604,A,0.972010,0.837953,3.22,Core,NaN,NaN,A992881
4,A992881:1:[DIRECT],365,4WZI,B,586,F,0.971867,0.807851,2.58,Protein interaction,A,NaN,A992881
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118711,P02920,53,2Y5Y,A,53,S,0.976684,0.925659,3.38,Core,NaN,NaN,LACY_ECOLI
118712,P02920,370,2Y5Y,A,370,G,0.976684,0.925659,3.38,Core,NaN,NaN,LACY_ECOLI
118713,P02920,367,2Y5Y,A,367,V,0.976684,0.925659,3.38,Surface,NaN,NaN,LACY_ECOLI
118714,P02920,333,2Y5Y,A,333,S,0.976684,0.925659,3.38,Core,NaN,NaN,LACY_ECOLI


In [29]:
struct[(struct['Input-ID']=='SL9A1_HUMAN')&(struct['Position']==169)&(
            struct['PDB-ID']=='6Z3Z')]

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
80014,P19634,169,6Z3Z,B,142,I,0.372283,0.442308,3.19,Surface,NaN,NaN,SL9A1_HUMAN
80017,P19634,169,6Z3Z,A,142,I,0.372283,0.442840,3.19,Core,NaN,NaN,SL9A1_HUMAN


In [18]:
struct.columns

Index(['Protein-ID', 'Position', 'PDB-ID', 'Chain', 'Residue-ID',
       'Residue AA type', 'Sequence identity', 'Coverage', 'Resolution',
       'Residue RIN simple classification', 'Chain type interaction partners',
       'Small molecules type interaction partners', 'Input-ID'],
      dtype='object')

In [24]:
struct[struct.duplicated(subset=[
    'Protein-ID', 'Position', 'PDB-ID', 'Residue RIN simple classification'
])].sort_values(by=[
    'Protein-ID',
    'PDB-ID',
    'Position',
])

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID
468631,A000082:1:BDH_HUMAN,115,2RHR,A,65,R,0.351931,0.636612,2.5,Surface,NaN,NaN,A000082
468630,A000082:1:BDH_HUMAN,288,2RHR,A,239,L,0.351931,0.636612,2.5,Core,NaN,NaN,A000082
468605,A000082:1:BDH_HUMAN,115,3EMK,D,62,S,0.358586,0.516971,2.5,Surface,NaN,NaN,A000082
468606,A000082:1:BDH_HUMAN,115,3EMK,B,62,S,0.373057,0.505236,2.5,Surface,NaN,NaN,A000082
468607,A000082:1:BDH_HUMAN,115,3ENN,D,62,S,0.355330,0.518421,2.1,Surface,NaN,NaN,A000082
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245581,R930770:1:SUBT_BACSU,324,6F9M,A,248,S,0.446970,0.643902,1.3,Core,NaN,NaN,R930770
245583,R930770:1:SUBT_BACSU,354,6F9M,A,283,A,0.446970,0.643902,1.3,Protein interaction,B,NaN,R930770
245492,R930770:1:SUBT_BACSU,209,6KHW,B,143,K,0.466165,0.648780,2.7,Surface,NaN,NaN,R930770
245491,R930770:1:SUBT_BACSU,324,6KHW,B,259,T,0.466165,0.648780,2.7,Surface,NaN,NaN,R930770


# check missing pdb_rins

In [86]:
df = pd.read_csv('../protein_positions_recommended_struct.tsv',sep='\t')
df.head()

,Uniprot_AC,Pos_A,Pos_C,PDB_ID,PDB_ID_A,PDB_ID_C
0,P63279,106,74,3UIP,3UIP,3UIP
1,P63279,106,67,4W5V,4W5V,4W5V
2,P63279,106,78,4W5V,4W5V,4W5V
3,P63279,106,40,4W5V,4W5V,4W5V
4,P63279,26,106,bad_struct,4W5V,3UIP


In [87]:
pdblst = df['PDB_ID_A'].values.tolist()
pdblst.extend(df['PDB_ID_C'].values.tolist())
pdblst = list(set(pdblst))
#pdblst

In [88]:
import os

In [89]:
opened = 0
nopdb = 0
to_download = []
path = '../../RINs_final/'
for elem in pdblst:
    elem = elem[:4]
    if os.path.isdir(path + elem.lower()):
       
        opened +=1
    else:
        nopdb +=1
        to_download.append(elem)
       
print(opened)
print(nopdb)

512
508


In [90]:
with open('missing_rins.txt', 'w') as f:
    for item in to_download:
        f.write("%s\n" % item)

# some drafts.

In [70]:
struct[struct['Protein-ID']=='ACM3_RAT']

,Protein-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners,Input-ID


In [11]:
struct1['Protein-ID'].value_counts()

P62593    62630
P63279    23182
P04637     8191
P03369     4417
P0A8V2     3222
P08799     2558
P04147     1117
P0A6F5      842
P08483      722
P0A1H3      662
P00877      564
P01100      478
P0A881      460
P46937      452
Q9ES00      416
P00410      413
O18209      410
Q2FJ93      387
P02920      384
P0AES6      369
P03023      353
P29255      246
P13508      132
P04610       75
P0AGA2       60
P12866       47
P0A9E5       27
P07017       17
P19634       12
Q07307        4
D6VTK4        3
Name: Protein-ID, dtype: int64

In [12]:
! pwd

/Users/azbukinanadezda/Google Диск/Ramensky_lab/Compensatory_project/Paired_mapping/annotation_tables 2


In [82]:
! ls

find_structure.ipynb                 paired_PMD_full_annotation_table.tsv
maveDB_full_annotation_table.tsv


In [87]:
prot_list = ['ACM3_RAT',
 'BLAT_ECOLX',
 'CH60_ECOLI',
 'COX2_YEAST',
 'EFG_SALTY',
 'EFG_STAA3',
 'FNR_ECOLI',
 'GLP1_CAEEL',
 'GYRB_ECOLI',
 'LACI_ECOLI',
 'LACY_ECOLI',
 'LACY_STRTR',
 'MCP2_ECOLI',
 'MELB_ECOLI',
 'MYS2_DICDI',
 'P53_HUMAN',
 'PMY13_CAEEL',
 'POL_HV1A2',
 'PSAB_SYNY3',
 'RBL_CHLRE',
 'RPOB_ECOLI',
 'SECY_ECOLI',
 'SL9A1_HUMAN',
 'STE2_YEAST',
 'STE6_YEAST',
 'TAT_HV1BR',
 'TCR3_ECOLX',
 'TRPR_ECOLI',
 'UAPA_EMENI',
 'VG05_BPP22',
 'VGA_BPPHX',
 'VGC_BPPHX',
 'VGF_BPPHX',
 'VGG_BPPHX',
 'VGH_BPPHX']

In [85]:
#out_alex = pd.read_csv('../../paired_mutations_CPD/paired_nonPMD.tsv',
                      #sep='\t')
    
out_alex = pd.read_csv('maveDB_full_annotation_table.tsv', sep='\t') 
out_alex.shape

(118716, 13)

In [151]:
out_alex[out_alex['Protein-ID']=='P03643']



,Protein-ID,Input-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners


In [89]:
out_alex.head()

,Protein-ID,Input-ID,Position,PDB-ID,Chain,Residue-ID,Residue AA type,Sequence identity,Coverage,Resolution,Residue RIN simple classification,Chain type interaction partners,Small molecules type interaction partners
0,Q07307,UAPA_EMENI,408,5I6C,A,408,Q,0.997917,0.836237,3.7,ligand interaction,NaN,A:1001
1,Q07307,UAPA_EMENI,528,5I6C,A,528,F,0.997917,0.836237,3.7,Core,NaN,NaN
2,Q07307,UAPA_EMENI,408,5I6C,B,408,Q,0.997863,0.815331,3.7,ligand interaction,NaN,B:1001
3,Q07307,UAPA_EMENI,528,5I6C,B,528,F,0.997863,0.815331,3.7,Core,NaN,NaN
4,P12866,STE6_YEAST,507,1MT0,A,607,S,0.408333,0.185902,2.6,Surface,NaN,NaN


In [88]:
for elem in prot_list:
    print(out_alex[out_alex['Protein-ID']==elem].shape)

(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
(0, 13)
